# PYTHON_APOSTAS

In [6]:
import pandas as pd
#import xlwings as xw
import openpyxl as op
import numpy as np
import datetime as dt
import numpy as np
import os

In [7]:
Months={"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
MonthsR={1:"Jan",2:"Feb",3:"Mar",4:"Apr",5:"May",6:"Jun",7:"Jul",8:"Aug",9:"Sep",10:"Oct",11:"Nov",12:"Dec"}

In [8]:
Trims={"Trim1":1,"Trim2":2,"Trim3":3,"Trim4":4}
TrimsR={1:"Trim1",2:"Trim2",3:"Trim3",4:"Trim4"}

In [9]:
Sems={"Sem1":1,"Sem2":2}
SemsR={1:"Sem1",2:"Sem2"}

### Funções

In [10]:
#dá-nos a data (em datetime) em função da linha x da coluna B
#file e sheet têm de estar em string
#só funciona se as datas estiverem na coluna "B", a partir da linha 2
#Podemos mudar a maneira como usamos o checkwrong date (ex: passar para parametro) para ser mais eficiente
def date(x,file,sheet,wrong_date=True): 
    import datetime as dt
    import xlwings as xw
    assert x>1 and isinstance(x,int) #o isinstance verifica se [0] é do tipo [1]
    d=xw.Book(file+".xlsx").sheets[sheet].cells(x,2).value
    if not(wrong_date):
        return d
    else:
        if isinstance(d, dt.datetime):
            return dt.datetime.strptime(str(d),"%Y-%d-%m %H:%M:%S")
        elif isinstance(d,str):
            if len(d)==10:
                    return dt.datetime.strptime(d,"%d/%m/%Y")
            elif len(d)==8:
                d=d[:6]+"20"+d[6:]
                return dt.datetime.strptime(d,"%d/%m/%Y")
            else: 
                print("Oops! Algo está a falhar na função date, qnd a data é uma string!")
        else:
            return "not str nor datetime"
    
#ATENÇÃO
#Esta função só sabe corrigir strings do tipo dd//mm//YY (ex: 23/04/19) para o séc.XXI
#Pq adiciona a str "20" para ficar dd/mm/20YY

In [11]:
#tds os parâmetros têm de ser em string
#idata e fdata (datas q delimitam o interv de tempo) têm de ser do tipo (dd/mm/aaaa)
#data_i é idata transformado em datetime, o mesmo para data_f
def date_range_str(idata,fdata,file,sheet): #dá um tuplo (a,b), em que "a" é primeira linha com jogos entre as datas e "b" a última
    import datetime as dt
    import pandas as pd
    import xlwings as xw
    import openpyxl as op
    if (isinstance(idata,dt.datetime) and isinstance(fdata,dt.datetime)):
        print("Como "+str(data_i)+" e "+str(data_f)+" estão em dt.datetime, usar função date_range()")
    if not (isinstance(idata,str) and isinstance(fdata,str)):
        print(str(idata)+" e "+str(idata)+" têm de estar em string")
    data_i=dt.datetime.strptime(idata,"%d/%m/%Y")
    data_f=dt.datetime.strptime(fdata,"%d/%m/%Y")
    #sheet_xw=xw.Book(file+".xlsx").sheets[sheet] #no futuro posso querer voltar a usar estas duas variáveis para 
    sheet_op=op.load_workbook(file+".xlsx")[sheet]  #tornar o programa mais rápido
    keep=True
    q=[]
    i=2
    while i<=sheet_op.max_row and keep:
        if date(i,file,sheet)>=data_i:
            q+=[i]
            k=i
            keep=False
        i+=1
    final=True
    while k<=(sheet_op.max_row) and final:
        if date(k,file,sheet)>data_f:
            q+=[k]
            final=False
        k+=1
    return (q[0]-2,q[1]-2)


#única diferença é q as datas já estão em datetime
def date_range(data_i,data_f,file,sheet): #dá um tuplo (a,b), em que "a" é primeira linha com jogos entre as datas e "b" a última
    import datetime as dt
    import pandas as pd
    import xlwings as xw
    import openpyxl as op
    if not (isinstance(data_i,dt.datetime) and isinstance(data_f,dt.datetime)):
        print(str(data_i)+" e "+str(data_f)+" têm de estar em dt.datetime")
    if (isinstance(data_i,str) and isinstance(data_f,str)):
        print("Como "+data_i+" e "+data_f+" são strings, usar a função date_range_str()")
    #sheet_xw=xw.Book(file+".xlsx").sheets[sheet] #no futuro posso querer voltar a usar estas duas variáveis para 
    sheet_op=op.load_workbook(file+".xlsx")[sheet]  #tornar o programa mais rápido
    keep=True
    q=[]
    i=2
    while i<sheet_op.max_row and keep:
        if date(i,file,sheet)>=data_i:
            q+=[i]
            k=i
            keep=False
        i+=1
    final=True
    while k<(sheet_op.max_row) and final:
        if date(k,file,sheet)>data_f:
            q+=[k]
            final=False
        k+=1
    if k==sheet_op.max_row:
        q+=[k+1]
    print("q",q)
    return (q[0]-2,q[1]-2)

#lembrar q o 2o índice não pertence de propóstio, pq na função q dá para escolher as colunas dos pandas o 2o parametro não é contado

In [12]:
#como eu defini q nas apostas a semana começa à sexta, isto faz com q se for 6a==1, 5a==7
def fix_week(x):
    return (((x.isoweekday()+2)%7)+1)


#damos o dia do ano (1-365) e o ano e devolve a data
#year e yday t~em de ser inteiros
def yday_date(year,yday): #damos o dia do ano (1-365) e o ano e devolve a data
    import datetime as dt
    return dt.datetime(year,1,1)+dt.timedelta(yday-1)

    
#função que dando um índice (número da semana) dá-nos o intervalo de dias em datetime
def week_to_dates(year,week_number):
    if year%4==0:
        tdays=366
    else:
        tdays=365 #assim temos o total de dias se for bissexto ou não
    first_day1=fix_week(dt.datetime.strptime("01/01/"+str(year),"%d/%m/%Y"))
    #first_day2=dt.datetime.strptime("01/01/"+year,"%d/%m/%Y").isoweekday()
    act_days=tdays-((8-first_day1)%7) #dá-nos os dias q temos do ano, sabendo q o primeiro dia será a primeira 6a 
    tweeks=act_days//7 #número total de semanas q vamos ter durante o ano
    first_day=tdays-act_days+1
    if act_days%7==0:
        if tweeks<week_number:
            print("ATENÇÃO!!! "+str(year)+" só tem "+str(tweeks)+" semanas!!!")
        return (dt.datetime(year,1,first_day)+dt.timedelta((7*(week_number-1))),dt.datetime(year,1,first_day)+dt.timedelta((7*(week_number))-1))
    else:
        if week_number<=tweeks:
            return (dt.datetime(year,1,first_day)+dt.timedelta((7*(week_number-1))),dt.datetime(year,1,first_day)+dt.timedelta((7*(week_number))-1))
        else:
            if week_number>(tweeks+1):
                print("ATENÇÃO!!! "+str(year)+" só tem "+str(tweeks+1)+" semanas!!! (a contar com o bocadinho de "+str(year+1)+")")
            special=7-(act_days-(7*(tweeks))) #número de dias q vamos comer ao ano seguinte na última semana do ano, pq tem de acabar numa 5a, nem q seja no ano seguinte
            return (dt.datetime(year,1,first_day)+dt.timedelta((7*(week_number-1))),dt.datetime(year+1,1,1)+dt.timedelta((special-1)))    

In [13]:
Months={"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12} #também está definido no início
#função que dando o mês dá-nos as datas desse mês
#month é em string
def month_to_dates(year,month):
    import datetime as dt
    m=Months[month]
    #first_day1=dt.datetime(year,m,1)
    found=False
    a=1
    q=[]
    while a<=10 and not(found): #neste ciclo, descobrimos o primeiro dia de mês q calha a uma 6a (dia em q começa o mês definido)
        dia1=dt.datetime(year,m,a)
        if fix_week(dia1)==1:
            q+=[dia1]
            found=True
        a+=1
    n=1
    found2=False
    while n<=6 and not(found2): #neste ciclo, descobrimos o primeiro dia do próximo mês (o dia antes é qnd acaba o mês em análise)
        dia2=q[0]+dt.timedelta(7*n)
        if q[0].month<12:
            if dia2.month==m+1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        elif q[0].month==12:
            if dia2.month==1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        else:
            print("Erro!!, os meses estão a ser superiores a 12")
        n+=1
    return q
    
    
#Mês é constituído por semanas
#Uma semana pertence a um mês se comecar(6a) nesse mês
#Por agora, é assim q vamos definir meses

In [14]:
def trim_to_dates(year,tri): #trimestre para dias
    t=Trims[tri]
    last=t*3
    first=last-2
    found=False
    a=1
    q=[]
    while a<=10 and not(found): #neste ciclo, descobrimos o primeiro dia de mês q calha a uma 6a (dia em q começa o mês definido)
        dia1=dt.datetime(year,first,a)
        if fix_week(dia1)==1:
            q+=[dia1]
            found=True
        a+=1
    a=1
    found=False
    while a<=10 and not(found): #neste ciclo, descobrimos o primeiro dia de mês q calha a uma 6a (dia em q começa o mês definido)
        dia1=dt.datetime(year,last,a)
        if fix_week(dia1)==1:
            abc=dia1
            found=True
        a+=1
    n=1
    found2=False
    while n<=6 and not(found2): #neste ciclo, descobrimos o primeiro dia do próximo mês (o dia antes é qnd acaba o mês em análise)
        dia2=abc+dt.timedelta(7*n)
        if abc.month<12:
            if dia2.month==last+1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        elif abc.month==12:
            if dia2.month==1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        else:
            print("Erro!!, os meses estão a ser superiores a 12")
        n+=1
    return q

#______________________________________________________________________________________________________________________________________________

def sem_to_dates(year,sem): #semestre para dias
    s=Sems[sem]
    last=s*6
    first=last-5
    found=False
    a=1
    q=[]
    while a<=10 and not(found): #neste ciclo, descobrimos o primeiro dia de mês q calha a uma 6a (dia em q começa o mês definido)
        dia1=dt.datetime(year,first,a)
        if fix_week(dia1)==1:
            q+=[dia1]
            found=True
        a+=1
    a=1
    found=False
    while a<=10 and not(found): #neste ciclo, descobrimos o primeiro dia de mês q calha a uma 6a (dia em q começa o mês definido)
        dia1=dt.datetime(year,last,a)
        if fix_week(dia1)==1:
            abc=dia1
            found=True
        a+=1
    n=1
    found2=False
    while n<=6 and not(found2): #neste ciclo, descobrimos o primeiro dia do próximo mês (o dia antes é qnd acaba o mês em análise)
        dia2=abc+dt.timedelta(7*n)
        if abc.month<12:
            if dia2.month==last+1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        elif abc.month==12:
            if dia2.month==1:
                q+=[dia2+dt.timedelta(-1)]
                found2=True
        else:
            print("Erro!!, os meses estão a ser superiores a 12")
        n+=1
    return q

In [15]:
sem_to_dates(2020,"Sem1")

[datetime.datetime(2020, 1, 3, 0, 0), datetime.datetime(2020, 7, 2, 0, 0)]

In [16]:
#mistura o date_range() com o week_to_dates(), month_to_dates() e trim/sem_to_dates(), indicando logo as linhas onde se encontram as datas da semana de índice "ind" 
#ind: semana(int)/mês(str)/trimestre(str)/semestre(str)
def dates_to_rows(year,ind,file,sheet):
    if isinstance(ind,int):
        qa=week_to_dates(year,ind)
    elif isinstance(ind,str):
        if ind[0]=="T":
            qa=trim_to_dates(year,ind)
        elif ind[0:3]=="Sem":
            qa=sem_to_dates(year,ind)
        else:
            qa=month_to_dates(year,ind)
    else:
        print("Está a dar Erro!!!!")
    print(qa)
    return date_range(qa[0],qa[1],file,sheet)

In [17]:
#cria um ficheiro novo e apaga o velho
#Boa função para quando fazemos updates com pandas e queremos gravar o ficheiro com updates e remover o antigo
#onv é a variável que contém as mudanças q fizemos ao ficheiro antigo
#Ficheiro antigo tem de estar fechado, senão o os não o elimina
def update_file(onv,old_name,new_name): 
    import pandas as pd
    import os
    print("Lembrar que o ficheiro a eliminar não pode estar aberto")
    onv.to_excel(new_name+".xlsx",index=False)
    os.remove(old_name+".xlsx")

In [18]:
#Esta função grava o ficheiro excel com várias folhas
#E o nome de cada folha fica como ("folha_%s" %n)

def save_xlsx_weeks(list_dfs,path):
    with pd.ExcelWriter(path+".xlsx") as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer, "WEEK_%s" %n +"_"+path, index=None)
        writer.save()
        
#EXEMPLO
#save_xlsX([prem_pd,df1],"Prem19_20.xlsx")

#______________________________________________________________________________________________________
#Rename de uma folha usando openpyxl
#o "path" não pode conter ".xlsx"
def rename_sheet(path,old_sheet,new_name):
    if not (isinstance(path,str) and isinstance(old_sheet,str) and isinstance(new_name,str)):
            print("Os argumentos têm de ser strings")
    import openpyxl as op
    print("Usamos openpyxl, os ficheiros têm de estar fechados")
    wb=op.load_workbook(path+".xlsx")
    sheet=wb[old_sheet]
    sheet.title=new_name
    wb.save(path+".xlsx")

In [19]:
#damos qnts semanas queremos de jogos e a função devolve uma DataFrame com essas x últimas jornadas/semanas
def last_games(week,path,sheet=0):
    file=pd.read_excel(path+'.xlsx',sheet)
    g=len(file.index)
    d=file.iloc[g-1]['Date']
    weekz=date_week(d.year,d)
    found=False
    i=1
    while i<g and not(found):
        day_=file.iloc[g-1-i]['Date']
        if weekz-(date_week(day_.year,day_))>=week: #se estamos na semana 10, e queremos as últimas 5 jornadas/semanas, queremos 6-10
            found=True
        else:
            i+=1
    return file[(g-i):(g)]

In [20]:
#ATENÇÃO esta função dá erro com datas do tipo '08/08/20' (sem 20)
#Precisamos do argumento year, pq definimos q há dias em 2019 do tipo '01/01/2020'

#damos o dia e o ano e devolve o número da semana desse ano
def date_week(year,date):
    import datetime as dt
    if isinstance(date,str):
        day_=dt.datetime.strptime(date,"%d/%m/%Y")
    elif isinstance(date,dt.datetime):
        day_=date
    else:
        print("Erro! Not Datetime nor string.")
    first_day=dt.datetime(year,1,((8-fix_week(dt.datetime.strptime("01/01/"+str(year),"%d/%m/%Y")))%7)+1) #primeiro dia do ano das apostas
    take=first_day.day #dias a tirar 
    act_day=(day_.timetuple().tm_yday)-take #dia do ano das apostas do primeiro dia da folha de excel
    final=(act_day//7)+1
    if final>0:
        return final
    else:
        return 52

#Nota q pode dar erros no futuro
#Qnd damos uma data de janeiro q ainda pertence à semana do ano passado, a função dá 0
#Já adicionei um if para resolver este probl

In [21]:
#as próximas funções fazem o mesmo q a de cima faz, mas dão o mês/trim/sem em vez do número da semana

def date_month(year,date):
    import datetime as dt
    if isinstance(date,str):
        day_=dt.datetime.strptime(date,"%d/%m/%Y")
    elif isinstance(date,dt.datetime):
        day_=date
    else:
        print("Erro! Not Datetime nor string.")
    weekday=fix_week(day_)
    n=weekday-1
    return MonthsR[(day_-dt.timedelta(n)).month]

In [22]:
def date_trim(year,date):
    import datetime as dt
    if isinstance(date,str):
        day_=dt.datetime.strptime(date,"%d/%m/%Y")
    elif isinstance(date,dt.datetime):
        day_=date
    else:
        print("Erro! Not Datetime nor string.")
    weekday=fix_week(day_)
    n=weekday-1
    return TrimsR[(((((day_-dt.timedelta(n)).month)-1)//3)+1)]
    
#__________________________________________________________________________________
    
def date_sem(year,date):
    import datetime as dt
    if isinstance(date,str):
        day_=dt.datetime.strptime(date,"%d/%m/%Y")
    elif isinstance(date,dt.datetime):
        day_=date
    else:
        print("Erro! Not Datetime nor string.")
    weekday=fix_week(day_)
    n=weekday-1
    return SemsR[(((((day_-dt.timedelta(n)).month)-1)//6)+1)]

In [23]:
date_sem(2019,"31/12/2019")

'Sem2'

In [24]:
#Se não me engano, devolve as semanas/meses/trim/sem de um ficheiro
def which_weeks(path,sheet):
    import datetime as dt
    import openpyxl as op
    wb_op=op.load_workbook(path+".xlsx")
    s1_op=wb_op[sheet]
    dia=date(2,path,sheet)
    year1=dia.year 
    week1=date_week(year1,dia)
    monthe=date_month(year1,dia)
    month1=Months[monthe]
    trime=date_trim(year1,dia)
    trim1=Trims[trime]
    seme=date_sem(year1,dia)
    sem1=Sems[seme]
    q=[year1,[seme,trime,monthe,week1]]
    i=2
    k=10
    maxi=s1_op.max_row
    while (i+k)<=maxi and not(1<=date_week(date(i+k,path,sheet).year,date(i+k,path,sheet))<=10):
        save=i+k
        bomdia=date_week(year1,date(i+k,path,sheet))
        bomdia2=date_month(year1,date(i+k,path,sheet))
        bomdia3=date_trim(year1,date(i+k,path,sheet))
        if Trims[bomdia3]==trim1+1:
            q[1]+=[bomdia3]
            trim1+=1
        if Months[bomdia2]==month1+1: #como, qnd chega à semana 1 do próximo ano, o ciclo fecha, o mês nunca deve chegar a Janeiro e dar Erro
            q[1]+=[bomdia2] #este if verifica/adiciona um mês
            month1+=1
        if bomdia==week1+1:
            q[1]+=[week1+1]
            week1+=1
            k+=10
        elif bomdia==week1:
            k+=10
        else:
            a=9
            while a>=0:
                boatarde3=date_trim(year1,date(i+k-a,path,sheet))
                boatarde2=date_month(year1,date(i+k-a,path,sheet))
                boatarde=date_week(year1,date(i+k-a,path,sheet))
                if Trims[boatarde3]>trim1:
                    q[1]+=[boatarde3]
                    trim1=Trims[boatarde3]
                if Months[boatarde2]>month1:
                    q[1]+=[boatarde2]
                    month1=Months[boatarde2]
                if boatarde>week1:
                    q[1]+=[boatarde]
                    week1=boatarde
                elif boatarde==bomdia:
                    a=-9
                a-=1
            k+=10
            week1=bomdia
    found=False
    while save<=maxi and not(found): #este ciclo serve para encontrar as variáveis com q vamos começar o ciclo do ano2
        if date(save,path,sheet).year!=year1:
            year2=date(save,path,sheet).year
            if 1<=date_week(year2,date(save,path,sheet))<=10:
                i=save
                monthe2=date_month(year2,date(save,path,sheet))
                trime2=date_trim(year2,date(save,path,sheet))
                seme2=date_sem(year2,date(save,path,sheet))
                week2=date_week(year2,date(save,path,sheet))
                found=True
        save+=1
    j=[year2,[seme2,trime2,monthe2,week2]]
    month2=Months[monthe2]
    trim2=Trims[trime2]
    k=10
    #este próximo ciclo é o mesmo do primeiro, mas para o 2o ano da época
    while (i+k)<=maxi:
        bomdia=date_week(year2,date(i+k,path,sheet))
        bomdia2=date_month(year2,date(i+k,path,sheet))
        bomdia3=date_trim(year2,date(i+k,path,sheet))
        if Trims[bomdia3]==trim2+1:
            j[1]+=[bomdia3]
            trim2+=1
        if Months[bomdia2]==month2+1: 
            j[1]+=[bomdia2]
            month2+=1
        if bomdia==week2+1:
            j[1]+=[week2+1]
            week2+=1
            if maxi-(i+k)>10:
                k+=10
            else:
                k+=1
        elif bomdia==week2:
            if maxi-(i+k)>10:
                k+=10
            else:
                k+=1
        else:
            a=9
            while a>=0:
                boatarde3=date_trim(year2,date(i+k-a,path,sheet))
                boatarde2=date_month(year2,date(i+k-a,path,sheet))
                boatarde=date_week(year2,date(i+k-a,path,sheet))
                if Trims[boatarde3]>trim2:
                    j[1]+=[boatarde3]
                    trim2=Trims[boatarde3]
                if Months[boatarde2]>month2:
                    j[1]+=[boatarde2]
                    month2=Months[boatarde2]
                if boatarde>week2:
                    j[1]+=[boatarde]
                    week2=boatarde
                elif boatarde==bomdia:
                    a=-9
                a-=1
            if maxi-(i+k)>10:
                k+=10
            else:
                k+=1
            week2=bomdia   
    return (q,j)

In [25]:
#escreve no ficheiro novas sheets com as semanas/meses/trim/sem

def major1(path,sheet):
    import pandas as pd
    import xlwings as xw
    import openpyxl as op
    import datetime as dt
    lista_semanas=which_weeks(path,sheet)
    info_ano1=lista_semanas[0]
    info_ano2=lista_semanas[1]
    q=[]
    for a in info_ano1[1]:
        (i,f)=dates_to_rows(info_ano1[0],a,path,sheet)
        excel=pd.read_excel(path+".xlsx")
        excel_next=excel[i:f]
        q+=[[excel_next,a]]
    for aa in info_ano2[1]:
        (i,f)=dates_to_rows(info_ano2[0],aa,path,sheet)
        excel=pd.read_excel(path+".xlsx")
        excel_next=excel[i:f]
        q+=[[excel_next,aa]]
    excel1=pd.read_excel(path+".xlsx")
    q=[[excel1,"ALL"]]+q
    premxw=xw.Book(path+".xlsx")
    premxw.close()
    with pd.ExcelWriter(path+".xlsx") as writer:
        for df in q:
            if isinstance(df[1],int):
                (df[0]).to_excel(writer, "WEEK%s" %(df[1]) +"_"+(fix_path(path)), index=None)
            elif isinstance(df[1],str):
                if (df[1])[0:3]=="Sem" or (df[1])[0]=="T" or (df[1])=="ALL":
                    (df[0]).to_excel(writer, (df[1])+"_"+(fix_path(path)), index=None)
                else:
                    (df[0]).to_excel(writer, "M%s" %(df[1])+"_"+(fix_path(path)), index=None)
    return writer.save()

In [26]:
#Importante para o nome das folhas, caso contrário, a folha ficava com o nome "C:\\Users\\joaom\\......."
def fix_path(path):
    found=False
    i=len(path)-1
    while i>=0 and not(found):
        if path[i]=='\\' or path[i]=='/':
            found=True
            k=i+1
        i-=1
    if found:
        return path[k:]
    else:
        return path

In [31]:
#colums_range('C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\___\\',0,4)

In [32]:
#agr as duas funções só adicionam as colunas, se elas ainda não existirem
#não adicionando colunas que ja lá estão, ficando com duas colunas iguais
def colums_range(path,sheet,pa):
    import numpy as np
    import pandas as pd
    import os
    for mn in range(pa,len(os.listdir(path))):
        file=pd.read_excel(path+os.listdir(path)[mn])
        ODDH=False
        ODDD=False
        ODDA=False
        Both=False
        Low=False
        Total=False
        Diff=False
        n=-7
        q=0
        while n<0: 
            check=False
            if file.columns[n]=="ODDH_Aver." and not(check):
                q+=1
                check=True
                ODDH=True
            if file.columns[n]=="ODDD_Aver." and not(check):
                q+=1
                check=True
                ODDD=True
            if file.columns[n]=="ODDA_Aver." and not(check):
                q+=1
                check=True
                ODDA=True
            if file.columns[n]=="Goal_Diff" and not(check):
                q+=1
                check=True
                Diff=True
            if file.columns[n]=="Goal_Total" and not(check):
                q+=1
                check=True
                Total=True
            if file.columns[n]=="Both_Scored" and not(check):
                q+=1
                check=True
                Both=True
            if file.columns[n]=="Lowest_Odd" and not(check):
                q+=1
                check=True
                Low=True
            n+=1
        if q==7:
            print("Já tem as colunas")
        else:
            if not(Diff):
                file["Goal_Diff"]=file["FTHG"]-file["FTAG"]
            if not(Total):
                file["Goal_Total"]=file["FTHG"]+file["FTAG"]
            found=False
            i=0
            while i<len(file.columns) and not(found) and not(ODDH):
                a=file.columns[i]
                if a=="BbAvH":
                    found=True
                    k=1
                if a=="AvgH":
                    found=True
                    k=0
                i+=1
            if found and k==1:
                file["ODDH_Aver."]=file["BbAvH"]
            elif found and k==0:
                file["ODDH_Aver."]=file["AvgH"]
        #
        #
        #
            while i<len(file.columns) and not(found) and not(ODDD):
                a=file.columns[i]
                if a=="BbAvD":
                    found=True
                    k=1
                if a=="AvgD":
                    found=True
                    k=0
                i+=1
            if found and k==1:
                file["ODDD_Aver."]=file["BbAvD"]
            elif found and k==0:
                file["ODDD_Aver."]=file["AvgD"]
        #
        #
        #
            while i<len(file.columns) and not(found) and not(ODDA):
                a=file.columns[i]
                if a=="BbAvA":
                    found=True
                    k=1
                if a=="AvgA":
                    found=True
                    k=0
                i+=1
            if found and k==1:
                file["ODDA_Aver."]=file["BbAvA"]
            elif found and k==0:
                file["ODDA_Aver."]=file["AvgA"]
            if not(Both):
                file["Both_Scored"]=np.where((file["FTHG"]>0)&(file["FTAG"]>0),"True","False")
            if not(Low):
                file["Lowest_Odd"]=np.where((file["ODDH_Aver."]<file["ODDA_Aver."])|(file["ODDH_Aver."]==file["ODDA_Aver."]),"H","A")
            q=[file]
            with pd.ExcelWriter(path+os.listdir(path)[mn]) as writer:
                for df in q:
                    df.to_excel(writer, "Sheet1", index=None)
            writer.save()

def colums_single(path,sheet):
    import pandas as pd
    import numpy as np
    file=pd.read_excel(path+".xlsx")
    ODDH=False
    ODDD=False
    ODDA=False
    Both=False
    Low=False
    Total=False
    Diff=False
    n=-7
    q=0
    while n<0: 
        check=False
        if file.columns[n]=="ODDH_Aver." and not(check):
            q+=1
            check=True
            ODDH=True
        if file.columns[n]=="ODDD_Aver." and not(check):
            q+=1
            check=True
            ODDD=True
        if file.columns[n]=="ODDA_Aver." and not(check):
            q+=1
            check=True
            ODDA=True
        if file.columns[n]=="Goal_Diff" and not(check):
            q+=1
            check=True
            Diff=True
        if file.columns[n]=="Goal_Total" and not(check):
            q+=1
            check=True
            Total=True
        if file.columns[n]=="Both_Scored" and not(check):
            q+=1
            check=True
            Both=True
        if file.columns[n]=="Lowest_Odd" and not(check):
            q+=1
            check=True
            Low=True
        n+=1
    if q==7:
        print("Já tem as colunas")
    else:
        if not(Diff):
            file["Goal_Diff"]=file["FTHG"]-file["FTAG"]
        if not(Total):
            file["Goal_Total"]=file["FTHG"]+file["FTAG"]
        found=False
        i=0
        while i<len(file.columns) and not(found) and not(ODDH):
            a=file.columns[i]
            if a=="BbAvH":
                found=True
                k=1
            if a=="AvgH":
                found=True
                k=0
            i+=1
        if found and k==1:
            file["ODDH_Aver."]=file["BbAvH"]
        elif found and k==0:
            file["ODDH_Aver."]=file["AvgH"]
        #
        #
        #
        while i<len(file.columns) and not(found) and not(ODDD):
            a=file.columns[i]
            if a=="BbAvD":
                found=True
                k=1
            if a=="AvgD":
                found=True
                k=0
            i+=1
        if found and k==1:
            file["ODDD_Aver."]=file["BbAvD"]
        elif found and k==0:
            file["ODDD_Aver."]=file["AvgD"]
        #
        #
        #
        while i<len(file.columns) and not(found) and not(ODDA):
            a=file.columns[i]
            if a=="BbAvA":
                found=True
                k=1
            if a=="AvgA":
                found=True
                k=0
            i+=1
        if found and k==1:
            file["ODDA_Aver."]=file["BbAvA"]
        elif found and k==0:
            file["ODDA_Aver."]=file["AvgA"]
        if not(Both):
            file["Both_Scored"]=np.where((file["FTHG"]>0)&(file["FTAG"]>0),"True","False")
        if not(Low):
            file["Lowest_Odd"]=np.where((file["ODDH_Aver."]<file["ODDA_Aver."])|(file["ODDH_Aver."]==file["ODDA_Aver."]),"H","A")
        q=[file]
        #xe=xw.Book(path+".xlsx") #fecha o ficheiro se estiver aberto
        #xe.close() #fecha o ficheiro se estiver aberto
        with pd.ExcelWriter(path+".xlsx") as writer:
            for df in q:
                df.to_excel(writer, "Sheet1", index=None)
        writer.save()            

In [40]:
colums_single('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//Serie_A___//I1',0)

In [ ]:
colums_single('PLBeast',0)

In [ ]:
#folhas é uma lista de listas das variáveirs associadas às novas folhas q queremos adicionar ([0]:variável,[1]:nome que queremos dar)
#a folha fica com o nome da variável
def add_sheets(path,folhas):
    import xlwings as xw
    import pandas as pd
    import openpyxl as op
    xe=xw.Book(path+".xlsx") #Com estas duas linhas conseguimos correr o programa
    xe.close() #mesmo que o ficheiro esteja aberto
    book = op.load_workbook(path+".xlsx")
    writer = pd.ExcelWriter(path+".xlsx", engine = 'openpyxl')
    writer.book = book
    for df in folhas:
        df[0].to_excel(writer, sheet_name = "%s"%df[1],index=None)
    writer.save()
    writer.close()
#Parece funcionar, a parte chata é um argumento ser uma lista de listas

In [ ]:
#Faz o major1 para os ficheiros de um destino
def major1_os(path,sheet,pa): #lembrar q o "pa" serve para indicar em q ficheiro começamos
    import os 
    for n in range(pa,len(os.listdir(path))):
        mekie=path+os.listdir(path)[n][:-5]
        colums_single(mekie,sheet)
        major1(mekie,sheet)
        print("___________________________CHECK________________________________")

In [ ]:
#adiciona ao ficheiro uma folha com a informação de apenas um clube
#com as colunas iguais á da sheet original

def add_club(path,club):
    import pandas as pd
    file=pd.read_excel(path+'.xlsx',0)
    x=file.loc[((file.HomeTeam==club)|(file.AwayTeam==club))]
    #if len(club)>=14:     #tive erros com o tamanho dos ficheiro, é para ver se isto resolve
    #    clu=strlong_converter(club)
    #else:
    #   clu=club
    print(club,'sheet')
    add_sheets(path,[[x,club]])

In [ ]:
#dá-nos a lista dos jogos q a equipa não ganhou,com:
#jogou em casa/fora,jornada(começa em 0),nome da outra equipa, perdeu/empatou, tinha q Odd para vencer, BothScored (T/F),+2.5 (T/F)

def aux_lostlist(path,club,game): 
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int):
        g=game
    else:
        g=len(file.index)
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            if (file.iloc[i]['FTR']=='H'):
                pass
            else:
                q+=[['HOME',i,file.iloc[i]["AwayTeam"],'D' if file.iloc[i]["FTR"]=='D' else 'L',file.iloc[i]['ODDH_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),not(file.iloc[i]['Goal_Total']<3)]]    
        else:
            if (file.iloc[i]['FTR']=='A'):
                pass
            else:
                q+=[['AWAY',i,file.iloc[i]["HomeTeam"],'D' if file.iloc[i]["FTR"]=='D' else 'L',file.iloc[i]['ODDA_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),not(file.iloc[i]['Goal_Total']<3)]]
    return q

In [ ]:
#adiciona a aux_lostlist ao excel
def lostlist(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_lostlist(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'FTR':[q[i][3]],'WinODD':[q[i][4]],'BothScored':[q[i][5]],'+2.5':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_LostList']])

In [ ]:
#dá-nos a lista dos jogos q a equipa ganhou,com:
#jogou em casa/fora,jornada(começa em 0),nome da outra equipa, W, tinha q Odd para vencer

def aux_winlist(path,club,game): 
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int):
        g=game
    else:
        g=len(file.index)
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            if (file.iloc[i]['FTR']=='D') or (file.iloc[i]['FTR']=='A'):
                pass
            else:
                q+=[['HOME',i,file.iloc[i]["AwayTeam"],'W',file.iloc[i]['ODDH_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),not(file.iloc[i]['Goal_Total']<3)]]    
        else:
            if (file.iloc[i]['FTR']=='H') or (file.iloc[i]['FTR']=='D'):
                pass
            else:
                q+=[['AWAY',i,file.iloc[i]["HomeTeam"],'W',file.iloc[i]['ODDA_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),not(file.iloc[i]['Goal_Total']<3)]]
    return q

In [ ]:
#adiciona a aux_winlist ao excel
def winlist(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_winlist(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'FTR':[q[i][3]],'WinODD':[q[i][4]],'BothScored':[q[i][5]],'+2.5':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_WinList']])

In [ ]:
#dá-nos a lista de jogos em q ambas as equipas marcaram,com:
#H/A, jornada (começa em 0), equipa com quem jogou, W/D/L, ODD para vencer 

def aux_BthScoredlist(path,club,game): 
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int):
        g=game
    else:
        g=len(file.index)
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            if abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total'])):
                pass
            else:
                q+=[['HOME',i,file.iloc[i]["AwayTeam"],'W' if file.iloc[i]['FTR']=='H' else 'D' if file.iloc[i]['FTR']=='D' else 'L',file.iloc[i]['ODDH_Aver.'],True,not(file.iloc[i]['Goal_Total']<3)]]    
        else:
            if abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total'])):
                pass
            else:
                q+=[['AWAY',i,file.iloc[i]["HomeTeam"],'W' if file.iloc[i]['FTR']=='A' else 'D' if file.iloc[i]['FTR']=='D' else 'L',file.iloc[i]['ODDA_Aver.'],True,not(file.iloc[i]['Goal_Total']<3)]]
    return q

In [ ]:
#adiciona a aux_BthScoredlist ao excel
def BthScoredlist(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_BthScoredlist(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'FTR':[q[i][3]],'WinODD':[q[i][4]],'BothScored':[q[i][5]],'+2.5':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_BthScoredList']])

In [ ]:
#dá-nos a lista de jogos em q o total de golos foi igual ou maior a 3,com:
#H/A, jornada (começa em 0), equipa com quem jogou, W/D/L, ODD para vencer 

def aux_list_25(path,club,game): 
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int): #assim, ao não lermos tds os dados, se quisermos, podemos usar esses para testar um modelo
        g=game 
    else:
        g=len(file.index)
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            if (file.iloc[i]['Goal_Total']<3):
                pass
            else:
                q+=[['HOME',i,file.iloc[i]["AwayTeam"],'W' if file.iloc[i]['FTR']=='H' else 'D' if file.iloc[i]['FTR']=='D' else 'L',file.iloc[i]['ODDH_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),True]]    
        else:
            if (file.iloc[i]['Goal_Total']<3):
                pass
            else:
                q+=[['AWAY',i,file.iloc[i]["HomeTeam"],'W' if file.iloc[i]['FTR']=='A' else 'D' if file.iloc[i]['FTR']=='D' else 'L',file.iloc[i]['ODDA_Aver.'],not(abs(file.iloc[i]['Goal_Diff'])==(abs(file.iloc[i]['Goal_Total']))),True]]
    return q

In [ ]:
#adiciona a aux_list_25 ao excel
def list_25(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_list_25(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'FTR':[q[i][3]],'WinODD':[q[i][4]],'BothScored':[q[i][5]],'+2.5':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_(+2.5)List']])

In [ ]:
#lista com os remates
def aux_shots_list(path,club,game):
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int): #assim, ao não lermos tds os dados, se quisermos, podemos usar esses para testar um modelo
        g=game 
    else:
        g=len(file.index)    
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            q+=[['HOME',i,file.iloc[i]['AwayTeam'],file.iloc[i]['ODDH_Aver.'],file.iloc[i]['HS'],file.iloc[i]['HST'],file.iloc[i]['FTHG']]]    
        else:
            q+=[['AWAY',i,file.iloc[i]['HomeTeam'],file.iloc[i]['ODDA_Aver.'],file.iloc[i]['AS'],file.iloc[i]['AST'],file.iloc[i]['FTAG']]]
    return q

def shots_list(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_shots_list(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'WinODD':[q[i][3]],'Shots':[q[i][4]],'ShotsTarget':[q[i][5]],'Goals':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_ShotsList']])

In [ ]:
def aux_cards_list(path,club,game):
    q=[]
    file=pd.read_excel(path+'.xlsx',club)
    if isinstance(game,int): #assim, ao não lermos tds os dados, se quisermos, podemos usar esses para testar um modelo
        g=game 
    else:
        g=len(file.index)    
    for i in range(0,g):
        if (file.iloc[i]['HomeTeam']==club):
            q+=[['HOME',i,file.iloc[i]['AwayTeam'],file.iloc[i]['ODDH_Aver.'],file.iloc[i]['HS'],file.iloc[i]['HST'],file.iloc[i]['FTHG'],file.iloc[i]['HY'],file.iloc[i]['HR']]]    
        else:
            q+=[['AWAY',i,file.iloc[i]['HomeTeam'],file.iloc[i]['ODDA_Aver.'],file.iloc[i]['AS'],file.iloc[i]['AST'],file.iloc[i]['FTAG'],file.iloc[i]['AY'],file.iloc[i]['AR']]]
    return q

def cards_list(path,club,game):
    file=pd.read_excel(path+'.xlsx',club)
    q=aux_cards_list(path,club,game)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'Jornada':[q[i][1]],'Team':[q[i][2]],'WinODD':[q[i][3]],'Shots':[q[i][4]],'ShotsTarget':[q[i][5]],'Goals':[q[i][6]],'Yellows':[q[i][7]],'Reds':[q[i][8]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if len(club)>=14:
        clu=strlong_converter(club)
    else:
        clu=club
    add_sheets(path,[[dta_pd,clu+'_CardsList']])

In [ ]:
#cria tds as listas para um clube (shots_list,lostlist,winlist,BthScoredlist,list_25)
#e cria uma lista com tds os jogos de um clube
def club_alllist(path,club,game):
    add_club(path,club)
    shots_list(path,club,game)
    lostlist(path,club,game)
    winlist(path,club,game)
    BthScoredlist(path,club,game)
    list_25(path,club,game)

In [ ]:
#dá uma lista com tds os clubes de uma época de uma liga
#teams é o número de equipas q tem a liga
def findall_clubs(path,teams):
    import pandas as pd
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    counter=0
    i=0
    done=False
    while i<len(file.index) and not(done):
        if i==teams+50:
            print('Está a ver demasiadas linhas, tem erro')
        if file.iloc[i]['HomeTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['HomeTeam']]
            counter+=1
        if file.iloc[i]['AwayTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['AwayTeam']]
            counter+=1
        if counter==teams:
            done=True
        i+=1
    return lista_c 

In [ ]:
#numa liga, cria tds as folhas para tds os clubes
#teams: qnts equipas tem a liga
#games: qnts jornadas queremos analisar
def major_clublist(path,teams,games):
    lista_c=findall_clubs(path,teams)
    print('vai começar o ciclo')
    for club in lista_c:
        print(club)
        club_alllist(path,club,games)

In [ ]:
#colums_single('LP19_20','Sheet1')

In [ ]:
#major_clublist('LP19_20',18,17)

In [ ]:
#cria uma lista de listas com tds as informações da infolist
def aux_infolist(path,row_i,row_f):
    import pandas as pd
    prem_pd=pd.read_excel(path+'.xlsx')
    clubs=[]
    for i in range(row_i,row_f):
        home=prem_pd.iloc[i]['HomeTeam']
        away=prem_pd.iloc[i]['AwayTeam']
        if home in locals():
            vars()[home][0]+=1 #número de jogos
            vars()[home][1]+=1 #número de jogos em casa, como joga em casa soma-se +1
            if prem_pd.iloc[i]['FTR']=='H':
                vars()[home][3]+=1
            elif prem_pd.iloc[i]['FTR']=='D':
                vars()[home][4]+=1
            else:
                vars()[home][5]+=1
            vars()[home][9]=vars()[home][3]+vars()[home][6] #total Win
            vars()[home][10]=vars()[home][4]+vars()[home][7] #total D
            vars()[home][11]=vars()[home][5]+vars()[home][8] #total Lost
            vars()[home][12]+=int(((prem_pd.iloc[i]['Goal_Total']+prem_pd.iloc[i]['Goal_Diff'])/2))  #Scored Home
            vars()[home][14]=vars()[home][12]+vars()[home][13] #total de golos marcados
            vars()[home][15]+=int(((prem_pd.iloc[i]['Goal_Total']-prem_pd.iloc[i]['Goal_Diff'])/2)) #Conceeded Home
            vars()[home][17]=vars()[home][15]+vars()[home][16]
            if not(abs(prem_pd.iloc[i]['Goal_Total'])==abs(prem_pd.iloc[i]['Goal_Diff'])):
                vars()[home][18]+=1 #jogo em casa em q ambas as equipas marcaram
            vars()[home][20]=vars()[home][18]+vars()[home][19] #jogos em q ambos marcaram
            if prem_pd.iloc[i]['Goal_Total']>=3: #jogos em casa q tiveram 3 golos ou mais
                vars()[home][21]+=1
            vars()[home][23]=vars()[home][21]+vars()[home][22]
        else:
            vars()[home]=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,home]
            vars()[home][0]+=1 #número de jogos
            vars()[home][1]+=1 #número de jogos em casa, como joga em casa soma-se +1
            if prem_pd.iloc[i]['FTR']=='H':
                vars()[home][3]+=1
            elif prem_pd.iloc[i]['FTR']=='D':
                vars()[home][4]+=1
            else:
                vars()[home][5]+=1
            vars()[home][9]=vars()[home][3]+vars()[home][6] #total Win
            vars()[home][10]=vars()[home][4]+vars()[home][7] #total D
            vars()[home][11]=vars()[home][5]+vars()[home][8] #total Lost
            vars()[home][12]+=int(((prem_pd.iloc[i]['Goal_Total']+prem_pd.iloc[i]['Goal_Diff'])/2))  #Scored Home
            vars()[home][14]=vars()[home][12]+vars()[home][13] #total de golos marcados
            vars()[home][15]+=int(((prem_pd.iloc[i]['Goal_Total']-prem_pd.iloc[i]['Goal_Diff'])/2)) #Conceeded Home
            vars()[home][17]=vars()[home][15]+vars()[home][16]
            if not(abs(prem_pd.iloc[i]['Goal_Total'])==abs(prem_pd.iloc[i]['Goal_Diff'])):
                vars()[home][18]+=1 #jogo em casa em q ambas as equipas marcaram
            vars()[home][20]=vars()[home][18]+vars()[home][19] #jogos em q ambos marcaram
            if prem_pd.iloc[i]['Goal_Total']>=3: #jogos em casa q tiveram 3 golos ou mais
                vars()[home][21]+=1
            vars()[home][23]=vars()[home][21]+vars()[home][22]
            clubs+=[vars()[home]]
        if away in locals():
            vars()[away][0]+=1 #número de jogos
            vars()[away][2]+=1 #número de jogos em casa, como joga em casa soma-se +1
            if prem_pd.iloc[i]['FTR']=='H':
                vars()[away][8]+=1 #vitória da equipa da casa é a derrota da equipa q joga fora
            elif prem_pd.iloc[i]['FTR']=='D':
                vars()[away][7]+=1
            else:
                vars()[away][6]+=1
            vars()[away][9]=vars()[away][3]+vars()[away][6] #total Win
            vars()[away][10]=vars()[away][4]+vars()[away][7] #total D
            vars()[away][11]=vars()[away][5]+vars()[away][8] #total Lost
            vars()[away][13]+=int(((prem_pd.iloc[i]['Goal_Total']-prem_pd.iloc[i]['Goal_Diff'])/2))  #Scored Away
            vars()[away][14]=vars()[away][12]+vars()[away][13] #total de golos marcados
            vars()[away][16]+=int(((prem_pd.iloc[i]['Goal_Total']+prem_pd.iloc[i]['Goal_Diff'])/2)) #Conceeded Away
            vars()[away][17]=vars()[away][15]+vars()[away][16]
            if not(abs(prem_pd.iloc[i]['Goal_Total'])==abs(prem_pd.iloc[i]['Goal_Diff'])):
                vars()[away][19]+=1 #jogo fora em q ambas as equipas marcaram
            vars()[away][20]=vars()[away][18]+vars()[away][19] #jogos em q ambos marcaram
            if prem_pd.iloc[i]['Goal_Total']>=3: #jogos fora q tiveram 3 golos ou mais
                vars()[away][22]+=1
            vars()[away][23]=vars()[away][21]+vars()[away][22]
        else:
            vars()[away]=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,away]
            vars()[away][0]+=1 #número de jogos
            vars()[away][2]+=1 #número de jogos em casa, como joga em casa soma-se +1
            if prem_pd.iloc[i]['FTR']=='H':
                vars()[away][8]+=1 #vitória da equipa da casa é a derrota da equipa q joga fora
            elif prem_pd.iloc[i]['FTR']=='D':
                vars()[away][7]+=1
            else:
                vars()[away][6]+=1
            vars()[away][9]=vars()[away][3]+vars()[away][6] #total Win
            vars()[away][10]=vars()[away][4]+vars()[away][7] #total D
            vars()[away][11]=vars()[away][5]+vars()[away][8] #total Lost
            vars()[away][13]+=int(((prem_pd.iloc[i]['Goal_Total']-prem_pd.iloc[i]['Goal_Diff'])/2))  #Scored Away
            vars()[away][14]=vars()[away][12]+vars()[away][13] #total de golos marcados
            vars()[away][16]+=int(((prem_pd.iloc[i]['Goal_Total']+prem_pd.iloc[i]['Goal_Diff'])/2)) #Conceeded Away
            vars()[away][17]=vars()[away][15]+vars()[away][16]
            if not(abs(prem_pd.iloc[i]['Goal_Total'])==abs(prem_pd.iloc[i]['Goal_Diff'])):
                vars()[away][19]+=1 #jogo fora em q ambas as equipas marcaram
            vars()[away][20]=vars()[away][18]+vars()[away][19] #jogos em q ambos marcaram
            if prem_pd.iloc[i]['Goal_Total']>=3: #jogos fora q tiveram 3 golos ou mais
                vars()[away][22]+=1
            vars()[away][23]=vars()[away][21]+vars()[away][22]
            clubs+=[vars()[away]]
    return clubs

In [ ]:
#coloca a infolist num ficheiro de excel
def infolist(path,row_i,row_f):
    print('Se quisermos a info toda, colocar str(tudo) no 3o argumento.')
    import pandas as pd
    if isinstance(row_f,int):
        last=row_f
    else:
        last=len(pd.read_excel(path+'.xlsx').index)
    omg=aux_infolist(path,row_i,last)
    lista_c=[]
    for i in range(0,len(omg)):
        dta=pd.DataFrame({'Club':[omg[i][24]],'Games':[omg[i][0]],'GamesHome':[omg[i][1]],'GamesAway':[omg[i][2]],'WinHome':[omg[i][3]],'DrawHome':[omg[i][4]],'LostHome':[omg[i][5]],'WinAway':[omg[i][6]],'DrawAway':[omg[i][7]],'LostAway':[omg[i][8]],'WinTotal':[omg[i][9]],'DrawTotal':[omg[i][10]],'LostTotal':[omg[i][11]],'ScoredHome':[omg[i][12]],'ScoredAway':[omg[i][13]],'ScoredTotal':[omg[i][14]],'ConceededHome':[omg[i][15]],'ConceededAway':[omg[i][16]],'ConceededTotal':[omg[i][17]],'BothScoredH':[omg[i][18]],'BothScoredA':[omg[i][19]],'BothScoredT':[omg[i][20]],'(+2.5)Home':[omg[i][21]],'(+2.5)Away':[omg[i][22]],'(+2.5)Total':[omg[i][23]]})
        lista_c+=[dta]
    dta_pd=pd.concat(lista_c)
    add_sheets(path,[[dta_pd,'InfoList']])

In [ ]:
#converte o nome de um clube longo para algo mais pequeno
def strlong_converter(a):
    counter=0
    done=False
    i=0
    while i<len(a) and not(done):
        if a[i].isupper():
            counter+=1
        if counter==2:
            done=True
        i+=1
    return a[:i]
            

In [ ]:
strlong_converter('Fortuna Dusseldorf')

In [ ]:
#delta define o intervalo de odds que vamos analisar
#com HA podemos escolher só analisar jogos em casa, jogos fora, ou ambos (pré-definido)
def by_odd_aux(odd,path,game,sheet=0,delta=0.1,HA='both'):
    q=[]
    file=pd.read_excel(path+'.xlsx',sheet)
    if isinstance(game,int): #assim, ao não lermos tds os dados, se quisermos, podemos usar esses para testar um modelo
        g=game 
    else:
        g=len(file.index)
    print(g)
    counter=0
    if HA=='both':
        for i in range(0,g):
            if odd<=round(file.iloc[i]['ODDH_Aver.'],2)<round(odd+delta,2):
                q+=[['HOME',round(file.iloc[i]['ODDH_Aver.'],2),file.iloc[i]['HomeTeam'],file.iloc[i]['AwayTeam'],'W' if file.iloc[i]['FTR']=='H' else 'D' if file.iloc[i]['FTR']=='D' else 'L',counter,1 if file.iloc[i]['FTR']=='H' else 0]]
                if file.iloc[i]['FTR']=='H':
                    counter+=1
                else:
                    counter=0
            elif odd<=round(file.iloc[i]['ODDA_Aver.'],2)<round(odd+delta,2):
                q+=[['AWAY',round(file.iloc[i]['ODDA_Aver.'],2),file.iloc[i]['HomeTeam'],file.iloc[i]['AwayTeam'],'W' if file.iloc[i]['FTR']=='A' else 'D' if file.iloc[i]['FTR']=='D' else 'L',counter,1 if file.iloc[i]['FTR']=='A' else 0]]
                if file.iloc[i]['FTR']=='A':
                    counter+=1
                else:
                    counter=0
        return q
    #Assim podemos escolher se queremos só a lista dos jogos em casa
    elif HA[0]=='H':
        for i in range(0,g):
            if odd<=round(file.iloc[i]['ODDH_Aver.'],2)<round(odd+delta,2):
                q+=[['HOME',round(file.iloc[i]['ODDH_Aver.'],2),file.iloc[i]['HomeTeam'],file.iloc[i]['AwayTeam'],'W' if file.iloc[i]['FTR']=='H' else 'D' if file.iloc[i]['FTR']=='D' else 'L',counter,1 if file.iloc[i]['FTR']=='H' else 0]]
                if file.iloc[i]['FTR']=='H':
                    counter+=1
                else:
                    counter=0
        return q
    #Assim podemos escolher se queremos só a lista dos jogos fora
    elif HA[0]=='A':
        for i in range(0,g):
            if odd<=round(file.iloc[i]['ODDA_Aver.'],2)<round(odd+delta,2):
                q+=[['AWAY',round(file.iloc[i]['ODDA_Aver.'],2),file.iloc[i]['HomeTeam'],file.iloc[i]['AwayTeam'],'W' if file.iloc[i]['FTR']=='A' else 'D' if file.iloc[i]['FTR']=='D' else 'L',counter,1 if file.iloc[i]['FTR']=='A' else 0]]
                if file.iloc[i]['FTR']=='A':
                    counter+=1
                else:
                    counter=0
        return q
    else:
        print('HA não está bem definido')
        return 'HA não está bem definido'
    
                
def by_odd(odd,path,game,sheet=0,delta=0.1,HA='both'):
    file=pd.read_excel(path+'.xlsx',sheet)
    q=by_odd_aux(odd,path,game,sheet=sheet,delta=delta,HA=HA) #ainda não tenho muita certeza se isto funciona (delta=delta)
    qw=[]
    for i in range(0,len(q)):
        dta=pd.DataFrame({'H/A':[q[i][0]],'ODD':[q[i][1]],'Home':[q[i][2]],'Away':[q[i][3]],'1X2':[q[i][4]],'Counter':[q[i][5]],'1Won/0NotWon':[q[i][6]]})
        qw+=[dta]
    dta_pd=pd.concat(qw)
    if HA=='both':
        add_sheets(path,[[dta_pd,str(odd)+'_to_'+str(round(odd+delta,2))]])
    elif HA[0]=='H':
        add_sheets(path,[[dta_pd,str(odd)+'_to_'+str(round(odd+delta,2))+'HOME']])
    elif HA[0]=='A':
        add_sheets(path,[[dta_pd,str(odd)+'_to_'+str(round(odd+delta,2))+'AWAY']])
    else:
        return 'Sth no right, HA não está bem definido'

In [ ]:
#by_odd(1.5,'E0','tudo',delta=0.05)

In [ ]:
#AINDA POR TESTAR
#coloca as datas todas em datetime
def all_datetime(path,sheet=0):
    file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    for it in range(0,limit):    
        if isinstance(file.iloc[it]['Date'],dt.datetime):
            file.at[it,'Date']=swap_datetime(file.iloc[it]['Date'])
        elif isinstance(file.iloc[it]['Date'],str):
            file.at[it,'Date']=dt.datetime.strptime(file.iloc[it]['Date'],"%d/%m/%Y")
    with pd.ExcelWriter(path+".xlsx") as writer:
        file.to_excel(writer, 'Sheet1', index=None)
        writer.save()

In [ ]:
#'arranja' o ficheiro para depois concatenar com outros ficheiros
#inclui o all_datetime, mas para não haver erros copiei a função para aqui
#o erro é que não eliminava a coluna 'Referee', pq o all_datetime vai buscar o ficheiro outra vez e não usa o q já não tem 'Referee'
def arrange_forc(path,sheet=0):
    print('Esta função precisa de ser atualizada, pq se o ficherio estiver aberto, não corre')
    colums_single(path,sheet) #vamos ver se não dá erro
    file=pd.read_excel(path+'.xlsx',0)
    if 'Referee' in file:
        file=file.drop(['Referee'],axis=1)   
    if check_wrongdate(file)[0]:      
        limit=len(file.index)
        for it in range(0,limit):
            #print(type(file.iloc[it]['Date']),it,file.iloc[it]['Date'])
            if isinstance(file.iloc[it]['Date'],dt.datetime):
                #print('ok2222222')
                file.at[it,'Date']=swap_datetime(file.iloc[it]['Date'])
            elif isinstance(file.iloc[it]['Date'],str):
                #print('ok')
                file.at[it,'Date']=dt.datetime.strptime(file.iloc[it]['Date'],"%d/%m/%Y")
    else:
        limit=len(file.index)
        for it in range(0,limit):
            if isinstance(file.iloc[it]['Date'],str):
                file.at[it,'Date']=dt.datetime.strptime(file.iloc[it]['Date'],"%d/%m/%Y")
    with pd.ExcelWriter(path+".xlsx") as writer:
        file.to_excel(writer, 'Sheet1', index=None)
        writer.save()

In [ ]:
#arrange_forc('BundTeste')

In [28]:
#serve para trocar o dia e mês, pq às vezes aparece trocado
def swap_datetime(date):
    return dt.datetime.strptime((dt.datetime.strftime(date,'%Y/%d/%m')),'%Y/%m/%d')

In [29]:
#Verifica se num ficheiro as datas estão trocadas
#o parametro é file, NÃO é path
def check_wrongdate(file,sheet=0):
    #file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    over_month=0
    month2=0
    counter=0
    counter_pd=0
    for i in range(1,limit): #ao começar no 1, temos sempre um valor anterior para comparar, se começasse no zero, comparava com o último valor [-1]
        day_=file.iloc[i]['Date']
        day2=file.iloc[i-1]['Date']
        if isinstance(day_,dt.datetime):
            counter+=1
            if isinstance(day2,str):
                if len(day2)==10:
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                elif len(day2)==8:                                        #serve para datas qnão têm '20' (2/3/15)
                    day2=day2[:6]+"20"+day2[6:]
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                else: 
                    print("Oops! Algo está a falhar na função date, qnd a data é uma string!")
                if ((day_.month)-(date_str.month)!=1) and ((day_.month)-(date_str.month)!=0):
                    over_month=1 #qnd muda de str para datetime, e estão trocados mes e dia a diferença entre os meses é estranha (diferente de 1)
            elif isinstance(day2,dt.datetime):
                if (day2.day==day_.day) and (day2.month!=day_.month):
                    #este if é para 'apanhar' qnd troca mes com dia, então o dia é igual, mas o mês muda em linhas consecutivas
                    month2=1
        elif isinstance(day_,pd.Timestamp):
            counter_pd+=1
    if (over_month+month2)==2:
        return (True, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp')
    else:
        return (False, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp',
               'over_month = '+str(over_month),
               'month2 = '+str(month2))
    

In [30]:
#check_wrongdate('BundTeste')

### ATENÇÃO
As funções antigas mudavam o datetime para ficar bem, como agr temos uma função q põe tudo em datetime,

###### Considerar mudar essa função
`date()`

In [54]:
#arrange_forc('BundTeste')

In [55]:
#w é a lista de ficheiros q queremos concatenar
#name é o nome que queremos dar ao ficheiro
def concat(w,name,sheet=0):
    q=[]
    for path in w:
        arrange_forc(path,sheet=sheet)
        oi=pd.read_excel(path+'.xlsx',0)
        q+=[oi]
    file=pd.concat(q,sort=False)
    file1=file.sort_values(['Date','Time'], ascending=[True,True])
    file1.to_excel(name+'.xlsx',index=False)

In [99]:
#concat(['BundTeste','PLTeste','LTeste','EredTeste','SATeste','LLTeste','LPTeste'],'19_20')

In [ ]:
def add_lion(fileN,fileC,clubs,SSTG=True,LowestLost=True,AwayLast=True,Surprise=True,conceed_aver=3,sstg=5,game_points=5,h_a=10,no_surprise=100,surprise_odd=1.4):
    #game_points: qntspontos teve nos último game_point jogos || també uso para saber qnts game_points jogou fora
    #no_surprise é o valor a atribuir aos jogos sem possibilidade de serem surpresa
    #surprise_odd é a odd a partir da qual já não é considerado surpresa
    
    limit=len(fileN.index)
    h=[]
    l=[]
    bigput_low=[[],[],[],[],[],[],[],[],[],[],[],[],[],[]]        #listas que vamos adicionar à df
    bigput_high=[[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    low_odd=[] #lista das odds + baixas
    high_odd=[] #lista das odds + altas
    HA=[] #equipa odd baixa casa(h_a)/fora(0)
    odds=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #odd counter (já vai ter counter negativos) (1,2.8,0.1)
    oddsput=[] #põe em coluna
    surprise=0 #counter de jogos surpresa
    surpriseput=[] #põe em coluna
    wonnotwon=[] #coluna Won/NotWon
    
    for club in clubs:                  # último da lista é o counter do clube (qnts jogos já jogou)
        vars()[club]=[[],[],[],[],0,0,0,0,0,0,0,10000,[],[],0] #cada clube tem uma lista de listas (shots/shotstarget/golos/concedidos/ganhos/jogos marca/jogos não concede)
    #[7]/[8]/[9]/[10] são as colunas de s/st/g/concd sem média (referem-se sempre ao jogo anterior)
    #[11] odd mais baixa que tinha e não ganhou (ninguém tem odd 10000, por isso é que lá deixei)
    #[12] pontos 
    #[13] jogos fora (+1fora/+0casa)
    for i in range(limit):                 
        if fileN.iloc[i]['Lowest_Odd']=='H': 
            l+=[fileN.iloc[i]['HomeTeam']]
            h+=[fileN.iloc[i]['AwayTeam']]
            LO=fileN.iloc[i]['HomeTeam']
            HO=fileN.iloc[i]['AwayTeam']
        else:
            l+=[fileN.iloc[i]['AwayTeam']]
            h+=[fileN.iloc[i]['HomeTeam']]
            HO=fileN.iloc[i]['HomeTeam']
            LO=fileN.iloc[i]['AwayTeam']
        
        #     #         #   Trata de criar colunas dos s/st/g/concd normais mas para LO e HO
        
        if vars()[LO][-1]==0:
            bigput_low[7]+=[0]
            bigput_low[8]+=[0]
            bigput_low[9]+=[0]
            bigput_low[10]+=[0]
        elif vars()[LO][-1]>0:
            bigput_low[7]+=[vars()[LO][7]]
            bigput_low[8]+=[vars()[LO][8]]
            bigput_low[9]+=[vars()[LO][9]]
            bigput_low[10]+=[vars()[LO][10]]
        
        if vars()[HO][-1]==0:
            bigput_high[7]+=[0]
            bigput_high[8]+=[0]
            bigput_high[9]+=[0]
            bigput_high[10]+=[0]
        elif vars()[HO][-1]>0:
            bigput_high[7]+=[vars()[HO][7]]
            bigput_high[8]+=[vars()[HO][8]]
            bigput_high[9]+=[vars()[HO][9]]
            bigput_high[10]+=[vars()[HO][10]]
        
        if fileN.iloc[i]['Lowest_Odd']=='H':
            vars()[LO][7]=fileN.iloc[i]['HS']
            vars()[LO][8]=fileN.iloc[i]['HST']
            vars()[LO][9]=fileN.iloc[i]['FTHG']
            vars()[LO][10]=fileN.iloc[i]['FTAG']
            vars()[HO][7]=fileN.iloc[i]['AS']
            vars()[HO][8]=fileN.iloc[i]['AST']
            vars()[HO][9]=fileN.iloc[i]['FTAG']
            vars()[HO][10]=fileN.iloc[i]['FTHG']
            
        elif fileN.iloc[i]['Lowest_Odd']=='A':
            vars()[LO][7]=fileN.iloc[i]['AS']
            vars()[LO][8]=fileN.iloc[i]['AST']
            vars()[LO][9]=fileN.iloc[i]['FTAG']
            vars()[LO][10]=fileN.iloc[i]['FTHG']
            vars()[HO][7]=fileN.iloc[i]['HS']
            vars()[HO][8]=fileN.iloc[i]['HST']
            vars()[HO][9]=fileN.iloc[i]['FTHG']
            vars()[HO][10]=fileN.iloc[i]['FTAG']
            
        #   #     #     #      #      #     #     #     #     #    # 
        #  #  #  Trata da média dos golos concedidos 
        
        if vars()[LO][-1]==0:
            bigput_low[3]+=[0] #se não jogos para fazer média, os primeiros valores são 0
            #neste caso é a média de golos concedidos
        elif 0<vars()[LO][-1]<conceed_aver:
            bigput_low[3]+=[sum(vars()[LO][3])/len(vars()[LO][3])]
        else:
            bigput_low[3]+=[sum(vars()[LO][3][(-(conceed_aver)):])/conceed_aver] 
        if vars()[HO][-1]==0:
            bigput_high[3]+=[0] #se não jogos para fazer média, os primeiros valores são 0
            #neste caso é a média de golos concedidos
        elif 0<vars()[HO][-1]<conceed_aver:
             bigput_high[3]+=[sum(vars()[HO][3])/len(vars()[HO][3])] 
        else:
            bigput_high[3]+=[sum(vars()[HO][3][(-(conceed_aver)):])/conceed_aver] 
            
        if fileN.iloc[i]['Lowest_Odd']=='H':  
            vars()[LO][3]+=[fileN.iloc[i]['FTAG']]
            vars()[HO][3]+=[fileN.iloc[i]['FTHG']]  
        
        elif fileN.iloc[i]['Lowest_Odd']=='A':
            vars()[HO][3]+=[fileN.iloc[i]['FTAG']]
            vars()[LO][3]+=[fileN.iloc[i]['FTHG']]
        
        # #    #    #   #    #    #   #    #    #    #    #  
        ## # Trata dos counter ganhos/marcou/concedeu
        
        if vars()[LO][-1]==0:
            bigput_low[4]+=[0] #no primeiro dia os counter estão tds a zero
            bigput_low[5]+=[0] 
            bigput_low[6]+=[0] 
        if vars()[HO][-1]==0:
            bigput_high[4]+=[0] #no primeiro dia os counter estão tds a zero
            bigput_high[5]+=[0] 
            bigput_high[6]+=[0]
            
            
        if vars()[LO][-1]>0:
            bigput_low[4]+=[vars()[LO][4]] 
            bigput_low[5]+=[vars()[LO][5]] 
            bigput_low[6]+=[vars()[LO][6]] 
        if vars()[HO][-1]>0:
            bigput_high[4]+=[vars()[HO][4]] 
            bigput_high[5]+=[vars()[HO][5]] 
            bigput_high[6]+=[vars()[HO][6]]
            
        #temos de dividir em dois casos: Odd+baixa Casa/Odd+baixa Fora    
        if fileN.iloc[i]['Lowest_Odd']=='H':   #Odd + baixa CASA 
            if fileN.iloc[i]['FTR']=='H': #se a equipa de casa ganha, como fica o counter
                if vars()[LO][4]>0:
                    vars()[LO][4]+=1 
                else:
                    vars()[LO][4]=1
                if vars()[HO][4]>0:
                    vars()[HO][4]=-1 
                else:
                    vars()[HO][4]-=1 
            if fileN.iloc[i]['FTR']=='A': #se a equipa de fora ganha, como fica o counter
                if vars()[LO][4]>0:
                    vars()[LO][4]=-1 
                else:
                    vars()[LO][4]-=1
                if vars()[HO][4]>0:
                    vars()[HO][4]+=1 
                else:
                    vars()[HO][4]=1 
            if fileN.iloc[i]['FTR']=='D': #se há empate, os counter de jogos ganhos vão todos a zero
                if vars()[LO][4]>0:
                    vars()[LO][4]=-1 
                else:
                    vars()[LO][4]-=1
                if vars()[HO][4]>0:
                    vars()[HO][4]=-1 
                else:
                    vars()[HO][4]-=1 
            if fileN.iloc[i]['FTHG']>0 and fileN.iloc[i]['FTAG']>0: # se ambas marcam
                #print('(+,+)')
                if vars()[LO][5]>0:
                    vars()[LO][5]+=1 
                else:
                    vars()[LO][5]=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]+=1    
                else:
                    vars()[HO][5]=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]=-1
                else:
                    vars()[LO][6]-=1
                if vars()[HO][6]>0:
                    vars()[HO][6]=-1 
                else:
                    vars()[HO][6]-=1 
            if fileN.iloc[i]['FTHG']>0 and fileN.iloc[i]['FTAG']==0: # se só casa marca
                #print('(+,0)')
                if vars()[LO][5]>0:
                    vars()[LO][5]+=1 
                else:
                    vars()[LO][5]=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]=-1    
                else:
                    vars()[HO][5]-=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]+=1
                else:
                    vars()[LO][6]=1
                if vars()[HO][6]>0:
                    vars()[HO][6]=-1 
                else:
                    vars()[HO][6]-=1 
            if fileN.iloc[i]['FTHG']==0 and fileN.iloc[i]['FTAG']>0: # se só fora marca
                #print('(0,+)')
                if vars()[LO][5]>0:
                    vars()[LO][5]=-1 
                else:
                    vars()[LO][5]-=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]+=1    
                else:
                    vars()[HO][5]=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]=-1
                else:
                    vars()[LO][6]-=1
                if vars()[HO][6]>0:
                    vars()[HO][6]+=1 
                else:
                    vars()[HO][6]=1 
            if fileN.iloc[i]['FTHG']==0 and fileN.iloc[i]['FTAG']==0: # se nenhuma marca
                #print('(0,0)')
                if vars()[LO][5]>0:
                    vars()[LO][5]=-1 
                else:
                    vars()[LO][5]-=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]=-1    
                else:
                    vars()[HO][5]-=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]+=1
                else:
                    vars()[LO][6]=1
                if vars()[HO][6]>0:
                    vars()[HO][6]+=1 
                else:
                    vars()[HO][6]=1 
        
        if fileN.iloc[i]['Lowest_Odd']=='A': 
            if fileN.iloc[i]['FTR']=='H': #se a equipa de casa ganha, como fica o counter
                if vars()[LO][4]>0:
                    vars()[LO][4]=-1 
                else:
                    vars()[LO][4]-=1
                if vars()[HO][4]>0:
                    vars()[HO][4]+=1 
                else:
                    vars()[HO][4]=1 
            if fileN.iloc[i]['FTR']=='A': #se a equipa de fora ganha, como fica o counter
                if vars()[LO][4]>0:
                    vars()[LO][4]+=1 
                else:
                    vars()[LO][4]=1
                if vars()[HO][4]>0:
                    vars()[HO][4]=-1 
                else:
                    vars()[HO][4]-=1 
            if fileN.iloc[i]['FTR']=='D': #se há empate, os counter de jogos ganhos vão todos a negativo
                if vars()[LO][4]>0:
                    vars()[LO][4]=-1 
                else:
                    vars()[LO][4]-=1
                if vars()[HO][4]>0:
                    vars()[HO][4]=-1 
                else:
                    vars()[HO][4]-=1 
            if fileN.iloc[i]['FTHG']>0 and fileN.iloc[i]['FTAG']>0: # se ambas marcam
                #print('(+,+)')
                if vars()[LO][5]>0:
                    vars()[LO][5]+=1 
                else:
                    vars()[LO][5]=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]+=1    
                else:
                    vars()[HO][5]=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]=-1
                else:
                    vars()[LO][6]-=1
                if vars()[HO][6]>0:
                    vars()[HO][6]=-1 
                else:
                    vars()[HO][6]-=1 
            if fileN.iloc[i]['FTHG']>0 and fileN.iloc[i]['FTAG']==0: # se só casa marca
                #print('(+,0)')
                if vars()[LO][5]>0:
                    vars()[LO][5]=-1 
                else:
                    vars()[LO][5]-=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]+=1    
                else:
                    vars()[HO][5]=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]=-1
                else:
                    vars()[LO][6]-=1
                if vars()[HO][6]>0:
                    vars()[HO][6]+=1 
                else:
                    vars()[HO][6]=1 
            if fileN.iloc[i]['FTHG']==0 and fileN.iloc[i]['FTAG']>0: # se só fora marca
                #print('(0,+)')
                if vars()[LO][5]>0:
                    vars()[LO][5]+=1 
                else:
                    vars()[LO][5]=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]=-1    
                else:
                    vars()[HO][5]-=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]+=1
                else:
                    vars()[LO][6]=1
                if vars()[HO][6]>0:
                    vars()[HO][6]=-1 
                else:
                    vars()[HO][6]-=1 
            if fileN.iloc[i]['FTHG']==0 and fileN.iloc[i]['FTAG']==0: # se nenhuma marca
                #print('(0,0)')
                if vars()[LO][5]>0:
                    vars()[LO][5]=-1 
                else:
                    vars()[LO][5]-=1    
                if vars()[HO][5]>0:
                    vars()[HO][5]=-1    
                else:
                    vars()[HO][5]-=1    
                if vars()[LO][6]>0:
                    vars()[LO][6]+=1
                else:
                    vars()[LO][6]=1
                if vars()[HO][6]>0:
                    vars()[HO][6]+=1 
                else:
                    vars()[HO][6]=1 
        
        
        
        
        #      #       #         #     #    #      #         #    
        # # #      Trata das médias de s/st/g
            
        if vars()[LO][-1]==0:
            bigput_low[0]+=[0] # os primeiros valores são 0
            bigput_low[1]+=[0] 
            bigput_low[2]+=[0] # neste caso é para o shots/shotstarget/goals
        elif 0<vars()[LO][-1]<sstg:
            bigput_low[0]+=[sum(vars()[LO][0])/len(vars()[LO][0])] #vai fazendo médio com os jogos q já existem
            bigput_low[1]+=[sum(vars()[LO][1])/len(vars()[LO][1])] 
            bigput_low[2]+=[sum(vars()[LO][2])/len(vars()[LO][2])]
        elif vars()[LO][-1]>=sstg:
            #print(vars()[LO])
            #print(bigput)
            bigput_low[0]+=[sum(vars()[LO][0][(-(sstg)):])/sstg] 
            bigput_low[1]+=[sum(vars()[LO][1][(-(sstg)):])/sstg] 
            bigput_low[2]+=[sum(vars()[LO][2][(-(sstg)):])/sstg] # neste caso é para o shots/shotstarget/goals
            
        #Agr é para a odd+alta
        if vars()[HO][-1]==0:
            bigput_high[0]+=[0] # os primeiros valores são 0
            bigput_high[1]+=[0] 
            bigput_high[2]+=[0] # neste caso é para o shots/shotstarget/goals
        elif 0<vars()[HO][-1]<sstg:
            bigput_high[0]+=[sum(vars()[HO][0])/len(vars()[HO][0])] #vai fazendo médio com os jogos q já existem
            bigput_high[1]+=[sum(vars()[HO][1])/len(vars()[HO][1])] 
            bigput_high[2]+=[sum(vars()[HO][2])/len(vars()[HO][2])]
        elif vars()[HO][-1]>=sstg:
            bigput_high[0]+=[sum(vars()[HO][0][(-(sstg)):])/sstg] 
            bigput_high[1]+=[sum(vars()[HO][1][(-(sstg)):])/sstg] 
            bigput_high[2]+=[sum(vars()[HO][2][(-(sstg)):])/sstg] # neste caso é para o shots/shotstarget/goals
        
        if fileN.iloc[i]['Lowest_Odd']=='H':   
            vars()[LO][0]+=[fileN.iloc[i]['HS']]
            vars()[LO][1]+=[fileN.iloc[i]['HST']]
            vars()[LO][2]+=[fileN.iloc[i]['FTHG']]
            vars()[HO][0]+=[fileN.iloc[i]['AS']]
            vars()[HO][1]+=[fileN.iloc[i]['AST']]
            vars()[HO][2]+=[fileN.iloc[i]['FTAG']]
        
        elif fileN.iloc[i]['Lowest_Odd']=='A': 
            vars()[LO][0]+=[fileN.iloc[i]['AS']]
            vars()[LO][1]+=[fileN.iloc[i]['AST']]
            vars()[LO][2]+=[fileN.iloc[i]['FTAG']]
            vars()[HO][0]+=[fileN.iloc[i]['HS']]
            vars()[HO][1]+=[fileN.iloc[i]['HST']]
            vars()[HO][2]+=[fileN.iloc[i]['FTHG']]

        #print(vars()[LO][6],LO)
          
          # # #  #   #    #     #      #       #       #       #   
        # # # Trata das listas da odds + altas mais baixas (coluna)
        if fileN.iloc[i]['Lowest_Odd']=='H':
            low_odd+=[fileN.iloc[i]['ODDH_Aver.']]
            high_odd+=[fileN.iloc[i]['ODDA_Aver.']]
        elif fileN.iloc[i]['Lowest_Odd']=='A':
            low_odd+=[fileN.iloc[i]['ODDA_Aver.']]
            high_odd+=[fileN.iloc[i]['ODDH_Aver.']]
        #   #    #    #    #    #     #     #     #     #     #
        
        ######## Trata da odd mais baixa com que o clube perdeu
        #if vars()[LO][-1]==0:
            #bigput_low[11]+=['To be defined']
        #elif vars()[LO][-1]>0:
        if fileN.iloc[i]['Lowest_Odd']=='H':
            #Equipa com odd + baixa (casa)
            if fileN.iloc[i]['ODDH_Aver.']<vars()[LO][11] and (fileN.iloc[i]['FTR']!='H'):
                if vars()[LO][-1]==0:
                    bigput_low[11]+=['To be defined']
                else:
                    bigput_low[11]+=[vars()[LO][11] if vars()[LO][11]!=10000 else 'To be defined']
                vars()[LO][11]=fileN.iloc[i]['ODDH_Aver.']
            else:
                if vars()[LO][-1]==0:
                    bigput_low[11]+=['To be defined']
                else:
                    bigput_low[11]+=[vars()[LO][11] if vars()[LO][11]!=10000 else 'To be defined']
            #Equipa com odd + alta (fora)
            if fileN.iloc[i]['ODDA_Aver.']<vars()[HO][11] and (fileN.iloc[i]['FTR']!='A'):
                if vars()[HO][-1]==0:
                    bigput_high[11]+=['To be defined']
                else:
                    bigput_high[11]+=[vars()[HO][11] if vars()[HO][11]!=10000 else 'To be defined']
                vars()[HO][11]=fileN.iloc[i]['ODDA_Aver.']
            else:
                if vars()[HO][-1]==0:
                    bigput_high[11]+=['To be defined']
                else:
                    bigput_high[11]+=[vars()[HO][11] if vars()[HO][11]!=10000 else 'To be defined']
                    
        
        #if vars()[HO][-1]==0:
            #bigput_high[11]+=['To be defined']
        #elif vars()[HO][-1]>0:
        if fileN.iloc[i]['Lowest_Odd']=='A':
            #Equipa com odd+ alta (casa)
            if fileN.iloc[i]['ODDH_Aver.']<vars()[HO][11] and (fileN.iloc[i]['FTR']!='H'):
                if vars()[HO][-1]==0:
                    bigput_high[11]+=['To be defined']
                else:
                    bigput_high[11]+=[vars()[HO][11] if vars()[HO][11]!=10000 else 'To be defined']
                vars()[HO][11]=fileN.iloc[i]['ODDH_Aver.']
            else:
                if vars()[HO][-1]==0:
                    bigput_high[11]+=['To be defined']
                else:
                    bigput_high[11]+=[vars()[HO][11] if vars()[HO][11]!=10000 else 'To be defined']
            #Equipa odd + baixa (fora)
            if fileN.iloc[i]['ODDA_Aver.']<vars()[LO][11] and (fileN.iloc[i]['FTR']!='A'):
                if vars()[LO][-1]==0:
                    bigput_low[11]+=['To be defined']
                else:
                    bigput_low[11]+=[vars()[LO][11] if vars()[LO][11]!=10000 else 'To be defined']
                vars()[LO][11]=fileN.iloc[i]['ODDA_Aver.']
            else:
                if vars()[LO][-1]==0:
                    bigput_low[11]+=['To be defined']
                else:
                    bigput_low[11]+=[vars()[LO][11] if vars()[LO][11]!=10000 else 'To be defined']
        
        #    #      #     #       #       #       #       #
        # Trata dos pontos nos últimos game_points
        if vars()[LO][-1]<game_points:
            bigput_low[12]+=[0]
        else:
            bigput_low[12]+=[sum(vars()[LO][12][(-(game_points)):])]
        
        if vars()[HO][-1]<game_points:
            bigput_high[12]+=[0]
        else:
            bigput_high[12]+=[sum(vars()[HO][12][(-(game_points)):])]
        
        if fileN.iloc[i]['Lowest_Odd']=='H':
            vars()[LO][12]+=[3 if fileN.iloc[i]['FTR']=='H' else 1 if fileN.iloc[i]['FTR']=='D' else 0]
            vars()[HO][12]+=[0 if fileN.iloc[i]['FTR']=='H' else 1 if fileN.iloc[i]['FTR']=='D' else 3]
        else:
            vars()[LO][12]+=[0 if fileN.iloc[i]['FTR']=='H' else 1 if fileN.iloc[i]['FTR']=='D' else 3]
            vars()[HO][12]+=[3 if fileN.iloc[i]['FTR']=='H' else 1 if fileN.iloc[i]['FTR']=='D' else 0]
            
        #    #        #         #     #      #     # 
       # Trata de dizer se a equipa com Odd + baixa joga Casa/Fora 
        if fileN.iloc[i]['Lowest_Odd']=='H':
            HA+=[h_a]
        else:
            HA+=[0]
       #####    #    #     #      #       #       #         #
    
        #    #    #   Trata do odd counter !  !   !   !      ! 
        
        
        if fileN.iloc[i]['Lowest_Odd']=='H':
            oddzz=round((fileN.iloc[i]['ODDH_Aver.'])*100)
            oddsput+=[odds[int((oddzz-100)//10)]]
        else:
            oddzz=round((fileN.iloc[i]['ODDA_Aver.'])*100)
            oddsput+=[odds[int((oddzz-100)//10)]]
                
        #counter
        if fileN.iloc[i]['Lowest_Odd']=='H':
            if fileN.iloc[i]['FTR']=='H': 
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]+=1
                else:
                    odds[int((oddzz-100)//10)]=1
            else: #se a equipa da casa odd + baixa não ganhar
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]=-1
                else:
                    odds[int((oddzz-100)//10)]-=1
                    
        else: # se a odd mais baixa for a equipa de fora
            if fileN.iloc[i]['FTR']=='H':
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]=-1
                else:
                    odds[int((oddzz-100)//10)]-=1
            else: #se a equipa de fora (odd + baixa ) ganhar
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]+=1
                else:
                    odds[int((oddzz-100)//10)]=1
    
    
    
        #   #    #   #   #       #      #      #  
        #   #    # Trata do counter das surpresas
        if vars()[LO][-1]==0:
            if fileN.iloc[i]['Lowest_Odd']=='H':
                if fileN.iloc[i]['ODDH_Aver.']<surprise_odd:
                    surpriseput+=[0]
                else:
                    surpriseput+=[no_surprise]
            # se a odd mais baixa for fora        
            else:
                if fileN.iloc[i]['ODDA_Aver.']<surprise_odd:
                    surpriseput+=[0]
                else:
                    surpriseput+=[no_surprise]
                
        elif vars()[LO][-1]>0:
            if fileN.iloc[i]['Lowest_Odd']=='H':
                if fileN.iloc[i]['ODDH_Aver.']<surprise_odd:
                    surpriseput+=[surprise]
                else:
                    surpriseput+=[no_surprise]
            # se a odd mais baixa for fora        
            else:
                if fileN.iloc[i]['ODDA_Aver.']<surprise_odd:
                    surpriseput+=[surprise]
                else:
                    surpriseput+=[no_surprise]
                    
        #counter
        if fileN.iloc[i]['Lowest_Odd']=='H':
            if fileN.iloc[i]['ODDH_Aver.']<surprise_odd:
                if fileN.iloc[i]['FTR']=='H':
                    if surprise>0:
                        surprise+=1
                    else:
                        surprise=1
                #se a equipa de casa (odd+baixa) não ganhar        
                else:
                    if surprise>0:
                        surprise=-1
                    else:
                        surprise-=1
        # se a odd mais baixa for fora        
        else:
            if fileN.iloc[i]['ODDA_Aver.']<surprise_odd:
                if fileN.iloc[i]['FTR']=='H':
                    if surprise>0:
                        surprise=-1
                    else:
                        surprise-=1
                #se a equipa de fora (odd+baixa) ganhar        
                else:
                    if surprise>0:
                        surprise+=1
                    else:
                        surprise=1
    
            
        # # # # # #   #    #   #   #   #   #   #
        # # # # Trata de saber qnts jogos teve fora nos últimos game_points jogos (+1 fora/+0 casa
        if vars()[LO][-1]<game_points:
            bigput_low[13]+=[0] #faz a média dos jogos já jogados, antes de chegar ao número de média pretendido
        else:
            bigput_low[13]+=[sum(vars()[LO][13][(-(game_points)):])]
            
        if vars()[HO][-1]==0:
            bigput_high[13]+=[0]
        elif 0<vars()[HO][-1]<game_points:
            bigput_high[13]+=[sum(vars()[HO][13])] #faz a média dos jogos já jogados, antes de chegar ao número de média pretendido
        else:
            bigput_high[13]+=[sum(vars()[HO][13][(-(game_points)):])]
            
        #preencher dados
        if fileN.iloc[i]['Lowest_Odd']=='H':
            vars()[LO][13]+=[0]
            vars()[HO][13]+=[1]
        else:
            vars()[LO][13]+=[1]
            vars()[HO][13]+=[0]
            
        #  #    #   #    #    #    #    #    #    #    # 
        #   # Trata do Won(1)/NotWon(0) em relação à odd + baixa
        if fileN.iloc[i]['Lowest_Odd']=='H':
            if fileN.iloc[i]['FTR']=='H':
                wonnotwon+=[1]
            else:
                wonnotwon+=[0]
        else:
            if fileN.iloc[i]['FTR']=='H':
                wonnotwon+=[0]
            else:
                wonnotwon+=[1]
            
        # # #    #   #   #   #   #   #   #  #   #   # 
        vars()[LO][-1]+=1 #faz o counter de cada clube andar
        vars()[HO][-1]+=1            
                
    fileC['Div']=fileN['Div']
    fileC['Date']=fileN['Date']
    fileC['Time']=fileN['Time']
    fileC['LO_Team']=l
    fileC['HO_Team']=h
    fileC['H/A']=HA
    if AwayLast:
        fileC['L_AwayLast_'+str(game_points)]=bigput_low[13]
        fileC['H_AwayLast_'+str(game_points)]=bigput_high[13]
    
    fileC['LowODD']=low_odd
    fileC['DrawODD']=fileN['ODDD_Aver.']
    fileC['HighODD']=high_odd
    if LowestLost:
        fileC['L_LowestLost']=bigput_low[11]
        fileC['H_LowestLost']=bigput_high[11]
    
    fileC['L_Points_'+str(game_points)]=bigput_low[12]
    fileC['H_Points_'+str(game_points)]=bigput_high[12]
    fileC['LoS'+str(sstg)]=bigput_low[0]
    fileC['HoS'+str(sstg)]=bigput_high[0]
    fileC['LoST'+str(sstg)]=bigput_low[1]
    fileC['HoST'+str(sstg)]=bigput_high[1]
    fileC['LoG'+str(sstg)]=bigput_low[2]
    fileC['HoG'+str(sstg)]=bigput_high[2]
    fileC['LoConceedAver_'+str(conceed_aver)]=bigput_low[3]
    fileC['HoConceedAver_'+str(conceed_aver)]=bigput_high[3]
    fileC['Lo_WinCount_']=bigput_low[4]
    fileC['Ho_WinCount_']=bigput_high[4]
    fileC['Lo_ScoresCount']=bigput_low[5]
    fileC['Ho_ScoresCount']=bigput_high[5]
    fileC['Lo_CleanSheet']=bigput_low[6]
    fileC['Ho_CleanSheet']=bigput_high[6]
    if SSTG:
        fileC['LoS']=bigput_low[7]
        fileC['HoS']=bigput_high[7]
        fileC['LoST']=bigput_low[8]
        fileC['HoST']=bigput_high[8]
        fileC['LoG']=bigput_low[9]
        fileC['HoG']=bigput_high[9]
        fileC['LoConcd']=bigput_low[10]
        fileC['HoConcd']=bigput_high[10]
        
    fileC['LeagueOddCounter']=oddsput
    if Surprise:
        fileC['SurprCount_'+str(surprise_odd)]=surpriseput
        
    fileC['Won/NotWon']=wonnotwon
    return fileC

In [2]:
def odd_allcounter(file):  #faz o count das odds qnd já juntámos várias ligas
    #Aqui não precisamos do ficheiro original, pq este já tem as odds
    
    limit=len(file.index)
    odds=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #odd counter (já vai ter counter negativos) (1,2.8,10)
    oddsput=[] #põe em coluna
    #for club in clubs:                 
    #    vars()[club]=[0] #agr só precisamos do counter do clube
    for i in range(limit):  
        oddzz = round((file.iloc[i]['LowODD'])*100)
        #põe os counter na lista para adicionar
        #print(i,'índice do ficheiro')
        #print(file.iloc[i]['LowODD'],'odd')
        #print(int((oddzz-100)//10),'indíce da lista')
        #print(oddzz,'odd*100')
        oddsput+=[odds[int((oddzz-100)//10)]]    

        #counter
        if file.iloc[i]['Won/NotWon']==1: 
            if odds[int((oddzz-100)//10)]>0:
                odds[int((oddzz-100)//10)]+=1
            else:
                odds[int((oddzz-100)//10)]=1
        else: #se a equipa da odd + baixa não ganhar
            if odds[int((oddzz-100)//10)]>0:
                odds[int((oddzz-100)//10)]=-1
            else:
                odds[int((oddzz-100)//10)]-=1
                
    file['ALLOddCounter']=oddsput
    return file    

In [1]:
def byodd_lion(path): #divide o ficheiro em folhas de odds
    file=pd.read_excel(path+'.xlsx',0)
    q=[[file]]
    for i in range(10,29,1):
        odd1=round(i/10,2)
        odd2=round(odd1+0.09,2)
        new=file.loc[(odd1<=file['LowODD']) & (file['LowODD']<=odd2)]
        q+=[[new,odd1,odd2]]
    with pd.ExcelWriter(path+".xlsx") as writer:
        for n, df in enumerate(q):
            if n==0:
                df[0].to_excel(writer, 'Lion_'+path, index=None)
            else:
                df[0].to_excel(writer, "%d_ODD_%s" %(df[1],df[2]) +"_"+path, index=None)
    writer.save()

_________________________


# Resto ---------------------------------------------------------------

In [142]:
add_club('PLTeste','Chelsea')

Chelsea sheet


In [145]:
shots_list('PLTeste','Chelsea','tudz')

In [104]:
oi1=pd.read_excel('BundTeste.xlsx',0)
oi2=pd.read_excel('PLTeste.xlsx',0)
#nice=pd.concat([oi1,oi2],sort=True)
#nice.to_excel('NICE.xlsx',index=False)

In [135]:
#nicez=pd.read_excel('NICE.xlsx',0)
by_odd(1.4,'19_20','tudo',HA=)

1765


In [413]:
pd.Timestamp.strptime('1/9/2019','%d/%m/%Y')

Timestamp('2019-09-01 00:00:00')

In [394]:
nice.iloc[30]['Date']

Timestamp('2019-09-14 00:00:00')

In [387]:
vars()['home']=[1,2,3]
a=vars()['home']
vars()['home']=[vars()['home'][0]+1,vars()['home'][1]+1,vars()['home'][2]+1]
home

[2, 3, 4]

In [222]:
vars()['home']=[vars()['home'][0]+1,vars()['home'][1],vars()['home'][2]]
home

[3, 3, 4]

In [197]:
prem_pd=pd.read_excel('E0.xlsx',0)
#premx=xw.Book('E0.xlsx')

In [2]:
q=[1,2,3]
def f():
    print('q' in globals())

In [3]:
f()

True


In [4]:
'q' in locals()

True

In [79]:
#função para eliminar o nome de tds os clubes de uma liga de variável
#team_number é o número de equipas dessa liga
#def delete_club_var(path,team_number):
 #   import pandas as pd
  #  file=pd.read_excel(path+'.xlsx')
   # done=False
    #i=0
    #counter=0
    #while i<100 and not(done):
    #    print(file.iloc[i]['HomeTeam'] in globals(),'aaaaaaaa')
     #   if file.iloc[i]['HomeTeam'] in globals() :
     #       del file.iloc[i]['HomeTeam']
      #      counter+=1
       # if file.iloc[i]['AwayTeam'] in globals():
       #     del file.iloc[i]['AwayTeam']
       #     counter+=1
       # if counter>=team_number:
       #     done=True
       # i+=1

In [10]:
Liverpool=[1,2,3,4,5,6,7]

In [34]:
West Ham

SyntaxError: invalid syntax (<ipython-input-34-ab2c5947e380>, line 1)

In [80]:
delete_club_var('E0',20)

True aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
True aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
True aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaaa
False aaaaaaa

In [82]:
 del Liverpool

In [336]:
del vars()['Liverpool']

KeyError: 'Liverpool'

In [333]:
Liverpool

NameError: name 'Liverpool' is not defined

In [26]:
int(2.0)

2

In [77]:
infolist('E0',0,200)

In [78]:
Liverpool

[1, 2, 3, 4, 5, 6, 7]

In [225]:
q=[1,2,3]
q[1]+=2+3
q

[1, 7, 3]

In [202]:
vars()['abc']=pd.read_excel('E0.xlsx',0)

In [203]:
abc

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,MaxCAHA,AvgCAHH,AvgCAHA,Goal_Diff,Goal_Total,ODDH_Aver.,ODDD_Aver.,ODDA_Aver.,Both_Scored,Lowest_Odd
0,E0,2019-09-08 00:00:00,20:00:00,Liverpool,Norwich,4,1,H,4,0,...,2.07,1.90,1.99,3,5,1.14,8.75,19.83,True,H
1,E0,2019-10-08 00:00:00,12:30:00,West Ham,Man City,0,5,A,0,1,...,1.98,1.97,1.92,-5,5,11.84,6.28,1.25,False,A
2,E0,2019-10-08 00:00:00,15:00:00,Bournemouth,Sheffield United,1,1,D,0,0,...,1.96,1.96,1.92,0,2,2.01,3.53,3.83,True,H
3,E0,2019-10-08 00:00:00,15:00:00,Burnley,Southampton,3,0,H,0,0,...,2.07,1.86,2.02,3,3,2.68,3.22,2.78,False,H
4,E0,2019-10-08 00:00:00,15:00:00,Crystal Palace,Everton,0,0,D,0,0,...,2.08,1.96,1.93,0,0,3.13,3.27,2.40,False,A
5,E0,2019-10-08 00:00:00,15:00:00,Watford,Brighton,0,3,A,0,1,...,1.91,2.05,1.84,-3,3,1.94,3.41,4.26,False,H
6,E0,2019-10-08 00:00:00,17:30:00,Tottenham,Aston Villa,3,1,H,0,1,...,1.87,2.08,1.80,2,4,1.30,5.53,10.51,True,H
7,E0,2019-11-08 00:00:00,14:00:00,Leicester,Wolves,0,0,D,0,0,...,1.98,2.06,1.84,0,0,2.22,3.28,3.48,False,H
8,E0,2019-11-08 00:00:00,14:00:00,Newcastle,Arsenal,0,1,A,0,0,...,1.97,2.00,1.89,-1,1,4.49,3.82,1.79,False,A
9,E0,2019-11-08 00:00:00,16:30:00,Man United,Chelsea,4,0,H,1,0,...,1.91,2.04,1.85,4,4,2.19,3.32,3.49,False,H


In [165]:
teste=pd.read_excel("E0.xlsx",'ManCity')

In [45]:
s1=premx.sheets["E0"]
s1.range('b2').value.year

2019

In [130]:
a=[]
a+=[[ManCity.iloc[1]["AwayTeam"],ManCity.iloc[1]["FTR"],ManCity.iloc[1]['ODDH_Aver.']]]
a

[['Tottenham', 'D', 1.36]]

In [136]:
q=[]
for i in range(0,len(ManCity.index)):
    if (ManCity.iloc[i]['HomeTeam']=='Man City'):
        if (ManCity.iloc[i]['FTR']=='H'):
            print("ok")
        else:
            q+=[['HOME',i,ManCity.iloc[i]["AwayTeam"],ManCity.iloc[i]["FTR"],ManCity.iloc[i]['ODDH_Aver.']]]    
    else:
        if (ManCity.iloc[i]['FTR']=='A'):
            print("ok2")
        else:
            q+=[['AWAY',i,ManCity.iloc[i]["HomeTeam"],ManCity.iloc[i]["FTR"],ManCity.iloc[i]['ODDA_Aver.']]]
q

ok2
ok2
ok
ok
ok2
ok2
ok
ok
ok
ok2
ok2
ok
ok
ok
ok2
ok2
ok
ok2


[['HOME', 1, 'Tottenham', 'D', 1.36],
 ['AWAY', 4, 'Norwich', 'H', 1.15],
 ['HOME', 7, 'Wolves', 'A', 1.13],
 ['AWAY', 11, 'Liverpool', 'H', 2.68],
 ['AWAY', 13, 'Newcastle', 'D', 1.18],
 ['HOME', 15, 'Man United', 'A', 1.31],
 ['AWAY', 18, 'Wolves', 'H', 1.4],
 ['HOME', 22, 'Crystal Palace', 'D', 1.1],
 ['AWAY', 24, 'Tottenham', 'H', 1.56],
 ['AWAY', 27, 'Man United', 'H', 1.64]]

In [5]:
date(22,"Prem19_20","E0").day

23

In [97]:
prem_op=op.load_workbook("Prem19_20.xlsx")
prem_op.create_sheet("teste")
prem_op.save("Prem19_20.xlsx")

In [99]:
sheet= op.load_workbook("Prem19_20"+".xlsx")["teste"]
sheet=prem_pd

In [9]:
date_range_str("3/10/2019","9/10/2019","Prem19_20","E0")

(72, 82)

In [10]:
isinstance(date(20,"Prem19_20","E0"),dt.date)

True

In [12]:
week_to_dates(2030,56)

ATENÇÃO!!! 2030 só tem 52 semanas!!! (a contar com o bocadinho de 2031)


(datetime.datetime(2031, 1, 24, 0, 0), datetime.datetime(2031, 1, 2, 0, 0))

In [26]:
prem_w7_2020=prem_pd[255:262] 

In [128]:
prem_pd["ODDH_Aver."]=((prem_pd["B365H"]+prem_pd["BWH"]+prem_pd["IWH"]+prem_pd["PSH"]+prem_pd["WHH"]+prem_pd["VCH"])/6)
prem_pd["ODDD_Aver."]=((prem_pd["B365D"]+prem_pd["BWD"]+prem_pd["IWD"]+prem_pd["PSD"]+prem_pd["WHD"]+prem_pd["VCD"])/6)
prem_pd["ODDA_Aver."]=((prem_pd["B365A"]+prem_pd["BWA"]+prem_pd["IWA"]+prem_pd["PSA"]+prem_pd["WHA"]+prem_pd["VCA"])/6)
prem_pd["Goal_Diff"]=(prem_pd["FTHG"]-prem_pd["FTAG"])
prem_pd["Lowest_Odd"]=np.where((prem_pd["ODDH_Aver."]<prem_pd["ODDA_Aver."])|(prem_pd["ODDH_Aver."]==prem_pd["ODDA_Aver."]),"H","A")

In [151]:
prem_pd.columns[4]

'AwayTeam'

In [129]:
save_xlsx_weeks([prem_pd],"PL19_20")

In [69]:
week_to_dates(2010,1)

(datetime.datetime(2019, 1, 4, 0, 0), datetime.datetime(2019, 1, 10, 0, 0))

In [160]:
date_week(2019,"03/01/2020")

0

In [200]:
which_weeks("Bund19_20","Sheet1")

??? 226


([2019, [33, 34, 35, 37, 38, 39, 40, 42, 43, 44, 45, 47, 48, 49, 50, 51]],
 [2020, [3, 4, 5, 6, 7, 8, 9, 10]])

In [142]:
major1("Bund19_20","Sheet1")

(datetime.datetime(2019, 8, 16, 0, 0), datetime.datetime(2019, 8, 22, 0, 0))
q [2, 11]
(datetime.datetime(2019, 8, 23, 0, 0), datetime.datetime(2019, 8, 29, 0, 0))
q [11, 20]
(datetime.datetime(2019, 8, 30, 0, 0), datetime.datetime(2019, 9, 5, 0, 0))
q [20, 29]
(datetime.datetime(2019, 9, 13, 0, 0), datetime.datetime(2019, 9, 19, 0, 0))
q [29, 38]
(datetime.datetime(2019, 9, 20, 0, 0), datetime.datetime(2019, 9, 26, 0, 0))
q [38, 47]
(datetime.datetime(2019, 9, 27, 0, 0), datetime.datetime(2019, 10, 3, 0, 0))
q [47, 56]
(datetime.datetime(2019, 10, 4, 0, 0), datetime.datetime(2019, 10, 10, 0, 0))
q [56, 65]
(datetime.datetime(2019, 10, 18, 0, 0), datetime.datetime(2019, 10, 24, 0, 0))
q [65, 74]
(datetime.datetime(2019, 10, 25, 0, 0), datetime.datetime(2019, 10, 31, 0, 0))
q [74, 83]
(datetime.datetime(2019, 11, 1, 0, 0), datetime.datetime(2019, 11, 7, 0, 0))
q [83, 92]
(datetime.datetime(2019, 11, 8, 0, 0), datetime.datetime(2019, 11, 14, 0, 0))
q [92, 101]
(datetime.datetime(2019, 11

In [29]:
prem_pd

NameError: name 'prem_pd' is not defined

In [314]:
fix_path(pat)

'SA05_06'

In [25]:
path="C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\Ligue_1__\\"

In [21]:
colums_single(path,"Sheet1")

In [27]:
major1_os(path,"Sheet1",4)

(datetime.datetime(2005, 7, 29, 0, 0), datetime.datetime(2005, 8, 4, 0, 0))
q [2, 12]
(datetime.datetime(2005, 8, 5, 0, 0), datetime.datetime(2005, 8, 11, 0, 0))
q [12, 22]
(datetime.datetime(2005, 8, 12, 0, 0), datetime.datetime(2005, 8, 18, 0, 0))
q [22, 32]
(datetime.datetime(2005, 8, 19, 0, 0), datetime.datetime(2005, 8, 25, 0, 0))
q [32, 43]
(datetime.datetime(2005, 8, 26, 0, 0), datetime.datetime(2005, 9, 1, 0, 0))
q [43, 52]
(datetime.datetime(2005, 9, 9, 0, 0), datetime.datetime(2005, 9, 15, 0, 0))
q [52, 62]
(datetime.datetime(2005, 9, 16, 0, 0), datetime.datetime(2005, 9, 22, 0, 0))
q [62, 82]
(datetime.datetime(2005, 9, 23, 0, 0), datetime.datetime(2005, 9, 29, 0, 0))
q [82, 92]
(datetime.datetime(2005, 9, 30, 0, 0), datetime.datetime(2005, 10, 6, 0, 0))
q [92, 101]
(datetime.datetime(2005, 10, 14, 0, 0), datetime.datetime(2005, 10, 20, 0, 0))
q [101, 111]
(datetime.datetime(2005, 10, 21, 0, 0), datetime.datetime(2005, 10, 27, 0, 0))
q [111, 121]
(datetime.datetime(2005, 10,

q [151, 162]
(datetime.datetime(2007, 12, 7, 0, 0), datetime.datetime(2007, 12, 13, 0, 0))
q [162, 172]
(datetime.datetime(2007, 12, 14, 0, 0), datetime.datetime(2007, 12, 20, 0, 0))
q [172, 182]
(datetime.datetime(2007, 12, 21, 0, 0), datetime.datetime(2007, 12, 27, 0, 0))
q [182, 191]
(datetime.datetime(2008, 1, 11, 0, 0), datetime.datetime(2008, 1, 17, 0, 0))
q [191, 201]
(datetime.datetime(2008, 1, 18, 0, 0), datetime.datetime(2008, 1, 24, 0, 0))
q [201, 221]
(datetime.datetime(2008, 1, 25, 0, 0), datetime.datetime(2008, 1, 31, 0, 0))
q [221, 231]
(datetime.datetime(2008, 2, 8, 0, 0), datetime.datetime(2008, 2, 14, 0, 0))
q [231, 241]
(datetime.datetime(2008, 2, 15, 0, 0), datetime.datetime(2008, 2, 21, 0, 0))
q [241, 251]
(datetime.datetime(2008, 2, 22, 0, 0), datetime.datetime(2008, 2, 28, 0, 0))
q [251, 261]
(datetime.datetime(2008, 2, 29, 0, 0), datetime.datetime(2008, 3, 6, 0, 0))
q [261, 271]
(datetime.datetime(2008, 3, 7, 0, 0), datetime.datetime(2008, 3, 13, 0, 0))
q [271, 

(datetime.datetime(2010, 4, 30, 0, 0), datetime.datetime(2010, 5, 6, 0, 0))
q [343, 361]
(datetime.datetime(2010, 5, 7, 0, 0), datetime.datetime(2010, 5, 13, 0, 0))
q [361, 372]
(datetime.datetime(2010, 5, 14, 0, 0), datetime.datetime(2010, 5, 20, 0, 0))
q [372, 382]
___________________________CHECK________________________________
(datetime.datetime(2010, 8, 6, 0, 0), datetime.datetime(2010, 8, 12, 0, 0))
q [2, 12]
(datetime.datetime(2010, 8, 13, 0, 0), datetime.datetime(2010, 8, 19, 0, 0))
q [12, 22]
(datetime.datetime(2010, 8, 20, 0, 0), datetime.datetime(2010, 8, 26, 0, 0))
q [22, 32]
(datetime.datetime(2010, 8, 27, 0, 0), datetime.datetime(2010, 9, 2, 0, 0))
q [32, 42]
(datetime.datetime(2010, 9, 10, 0, 0), datetime.datetime(2010, 9, 16, 0, 0))
q [42, 52]
(datetime.datetime(2010, 9, 17, 0, 0), datetime.datetime(2010, 9, 23, 0, 0))
q [52, 62]
(datetime.datetime(2010, 9, 24, 0, 0), datetime.datetime(2010, 9, 30, 0, 0))
q [62, 72]
(datetime.datetime(2010, 10, 1, 0, 0), datetime.dateti

q [142, 152]
(datetime.datetime(2012, 12, 7, 0, 0), datetime.datetime(2012, 12, 13, 0, 0))
q [152, 172]
(datetime.datetime(2012, 12, 14, 0, 0), datetime.datetime(2012, 12, 20, 0, 0))
q [172, 182]
(datetime.datetime(2012, 12, 21, 0, 0), datetime.datetime(2012, 12, 27, 0, 0))
q [182, 192]
(datetime.datetime(2013, 1, 11, 0, 0), datetime.datetime(2013, 1, 17, 0, 0))
q [192, 202]
(datetime.datetime(2013, 1, 18, 0, 0), datetime.datetime(2013, 1, 24, 0, 0))
q [202, 212]
(datetime.datetime(2013, 1, 25, 0, 0), datetime.datetime(2013, 1, 31, 0, 0))
q [212, 222]
(datetime.datetime(2013, 2, 1, 0, 0), datetime.datetime(2013, 2, 7, 0, 0))
q [222, 232]
(datetime.datetime(2013, 2, 8, 0, 0), datetime.datetime(2013, 2, 14, 0, 0))
q [232, 242]
(datetime.datetime(2013, 2, 15, 0, 0), datetime.datetime(2013, 2, 21, 0, 0))
q [242, 252]
(datetime.datetime(2013, 2, 22, 0, 0), datetime.datetime(2013, 2, 28, 0, 0))
q [252, 262]
(datetime.datetime(2013, 3, 1, 0, 0), datetime.datetime(2013, 3, 7, 0, 0))
q [262, 27

(datetime.datetime(2015, 4, 17, 0, 0), datetime.datetime(2015, 4, 23, 0, 0))
q [321, 331]
(datetime.datetime(2015, 4, 24, 0, 0), datetime.datetime(2015, 4, 30, 0, 0))
q [331, 342]
(datetime.datetime(2015, 5, 1, 0, 0), datetime.datetime(2015, 5, 7, 0, 0))
q [342, 352]
(datetime.datetime(2015, 5, 8, 0, 0), datetime.datetime(2015, 5, 14, 0, 0))
q [352, 362]
(datetime.datetime(2015, 5, 15, 0, 0), datetime.datetime(2015, 5, 21, 0, 0))
q [362, 372]
(datetime.datetime(2015, 5, 22, 0, 0), datetime.datetime(2015, 5, 28, 0, 0))
q [372, 382]
___________________________CHECK________________________________
(datetime.datetime(2015, 8, 7, 0, 0), datetime.datetime(2015, 8, 13, 0, 0))
q [2, 12]
(datetime.datetime(2015, 8, 14, 0, 0), datetime.datetime(2015, 8, 20, 0, 0))
q [12, 22]
(datetime.datetime(2015, 8, 21, 0, 0), datetime.datetime(2015, 8, 27, 0, 0))
q [22, 32]
(datetime.datetime(2015, 8, 28, 0, 0), datetime.datetime(2015, 9, 3, 0, 0))
q [32, 42]
(datetime.datetime(2015, 9, 11, 0, 0), datetime.d

q [152, 162]
(datetime.datetime(2017, 12, 8, 0, 0), datetime.datetime(2017, 12, 14, 0, 0))
q [162, 172]
(datetime.datetime(2017, 12, 15, 0, 0), datetime.datetime(2017, 12, 21, 0, 0))
q [172, 192]
(datetime.datetime(2018, 1, 12, 0, 0), datetime.datetime(2018, 1, 18, 0, 0))
q [192, 212]
(datetime.datetime(2018, 1, 19, 0, 0), datetime.datetime(2018, 1, 25, 0, 0))
q [212, 222]
(datetime.datetime(2018, 1, 26, 0, 0), datetime.datetime(2018, 2, 1, 0, 0))
q [222, 232]
(datetime.datetime(2018, 2, 2, 0, 0), datetime.datetime(2018, 2, 8, 0, 0))
q [232, 241]
(datetime.datetime(2018, 2, 9, 0, 0), datetime.datetime(2018, 2, 15, 0, 0))
q [241, 251]
(datetime.datetime(2018, 2, 16, 0, 0), datetime.datetime(2018, 2, 22, 0, 0))
q [251, 262]
(datetime.datetime(2018, 2, 23, 0, 0), datetime.datetime(2018, 3, 1, 0, 0))
q [262, 272]
(datetime.datetime(2018, 3, 2, 0, 0), datetime.datetime(2018, 3, 8, 0, 0))
q [272, 282]
(datetime.datetime(2018, 3, 9, 0, 0), datetime.datetime(2018, 3, 15, 0, 0))
q [282, 293]
(d

# --------------------------------------------------------------------------------------------------------------

# MLearning

In [1]:
from sklearn.datasets.samples_generator import make_regression
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

### LinRegression

In [2]:
def cal_cost(theta,X,y):
    m=len(y)
    predictions=X.dot(theta)
    cost=(1/2*m)*np.sum(np.square(predictions-y))
    return cost

#dim é o número de coeficientes a contar com o intercept
def gradient_descent(panda_df,learning_rate=0.01,iterations=10000):
    '''
    X = matriz j,2. cada j linha é [1,xj] sendo xj a abcissa de um valor  
    y = vetor [[y1],[y2],...,[yj]], yj o valor da variável dep. num ponto
    theta = vetor de thetas
    
    Returns the final theta vector and array of cost history over number of iterations
    '''
    import numpy as np
    X=adapt_X(panda_df)[0]
    y=adapt_X(panda_df)[1]
    dim=len(X[0])
    m=len(y)
    theta=np.random.randn(dim,1) #assim não temos de definir theta
    assert len(X[0])==dim #verificar se nos esquecemos de adicionar uma variável alguma dimensão
    cost_history=np.zeros(iterations)
    theta_history=np.zeros((iterations,dim))
    for it in range(iterations):
        
        prediction=np.dot(X,theta)
        theta=theta -(1/m)*learning_rate*(X.T.dot((prediction-y))) #theta = theta + (-gradiente * learning_rate)
        theta_history[it,:]=theta.T                          #learning_rate é como se fossem os passos q damos para descer, qnt mais pequeno mais preciso
        cost_history[it]=cal_cost(theta,X,y)                 #gradiente é o gradiente da cost_function (cálculos no caderno de Pt)
        
    return theta, cost_history[-1] #, theta_history

#### LogRegression

In [3]:
#theta vetor coluna
#X, matriz vetores linhas
def pred2(theta,X):
    return np.dot(X,theta)

def h(theta,X):
    e=np.e
    return 1/(1+(e**(-pred2(theta,X))))

#y é vetor linha
def LogR_cost(X,y,theta):
    m=len(y)
    return (-1/m)*(np.dot(y,np.log(h(theta,X)))+np.dot((1-y),(np.log(1-h(theta,X)))))

def LogR_grad(X,y,theta):
    m=len(y)
    return (1/m)*(np.dot(X.T,(h(theta,X)-(y.T))))

def gradient_descent_logit(panda_df,learning_rate=0.01,iterations=6000):
    import numpy as np
    X=adapt_X(panda_df,logit=True)[0]
    y=adapt_X(panda_df,logit=True)[1]
    m=len(y)
    dim=len(X[0])
    #print('Dim',dim)
    theta=np.random.randn(dim,1) #assim não temos de definir theta
    for it in range(iterations):
        cost=LogR_cost(X,y,theta)
        #print(X,y,theta)
        #print('________')
        #print(cost)
        theta=theta-LogR_grad(X,y,theta)*learning_rate
    return theta, cost
#Nota, parece q o cost às vezes dá erro, pq o valor da probabilidade chega a ser 1, dps faz-se Log(1-(1))=Log(0)=-inf
#Mas parece q a função não falha


#esta função ainda é beta-version, mas diz-nos a prob através do gráfico q aprendeu
#input_ pode ser array de linhas, cada linha, uma variável indep. 
def LogReg(input_,panda_df,learning_rate=0.01,iterations=6000):
    e=np.e
    G=gradient_descent_logit(panda_df,learning_rate=0.01,iterations=6000)
    t=G[0][0]+(np.dot(G[0][1:],input_))
    return 1/(1+(e**(-t)))

In [4]:
119%10

9

In [5]:
#limite diz a partir de q número se considera q acontece 1 (Logistic Regression) 
def confusion_matrix_LogReg(input_,panda_df,limite=0.64):
    trainz=input_.drop(input_.columns[-1],axis=1)
    check=adapt_X(input_,logit=True)[1] #assim temos o vetor y como queremos
    val=LogReg(trainz.T,panda_df)
    r=len(val[0])
    assert r==len(check[0])
        cmat=np.array([[0,0],[0,0]])   #confusion matrix
    for i in range(r):
        hj=(val[0][i]>limite) #hj True se for maior o valor a prob for maior q o limite e False caso contrário
        if hj:
            what=1
        else:
            what=0
        if what==1 and what==check[0][i]:
            cmat[0][0]+=1
        elif what==1 and what!=check[0][i]:
            cmat[0][1]+=1
        elif what==0 and what==check[0][i]:
            cmat[1][1]+=1
        elif what==0 and what!=check[0][i]:
            cmat[1][0]+=1
        else:
            print('Erro em confusion_matrix_LogReg')
    return cmat

#faz ciclo das várias confusion matrix variando o limite
#begin: número em q começa o limite, end: onde acaba
#delta: a diferença entre cada limite
#var é a variável que queremos analisar
def bestlimite_LogReg1(input_,panda_df,var,delta=0.01,begin=0.05,end=0.95):
    tot=round(end-begin,2)
    if (tot*10000)%(delta*10000)!=0:  #assim não temos float só se delta/end tiverem 5 casas decimais (0.00001)
        return 'end value não pode ser este'
    it=int((tot*10000)/(delta*10000))
    q=[]
    for i in range(0,it):
        pred=pred_percent(confusion_matrix_LogReg(input_,panda_df,limite=round((begin+i*delta),2)),var)
        if isinstance(pred,str):
            q+=[-1]
        else:
            q+=[pred]
    m=max(q)
    print(m, 'max_percent')
    print('_______________________________________________________')
    w = [u for u,j in enumerate(q) if j==m] #lista dos indices com o melhor pred_percent
    finalz=[]
    for e in w:
        finalz+=[[confusion_matrix_LogReg(input_,panda_df,limite=round((begin+e*delta),2)),round((begin+e*delta),2)]]
    print(finalz[0],'best pred_percents')
    print('-------------------------------------------------------')
    return finalz
#devolve lista dos limites com melhor resultado


#dá a soma das conf_matrix da LOgit_reg escolhendo o melhor limite para cada fold
def pre_cross_validation(panda_df,var,folds=10,delta=0.01,begin=0.05,end=0.95):
    g=len(panda_df.index)
    print(g)
    assert g>=folds
    extra=g%folds
    lastz=np.array([[0,0],[0,0]])
    for i in range(folds):
        if extra==0:
            test=panda_df[(0+i*folds):(folds+i*folds)]
        else:
            test=pd.concat([panda_df[(0+i*folds):(folds+i*folds)],panda_df[-extra:]],sort=False) 
        ind=list(test.index.values)
        train=panda_df.drop(ind)
        print('iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii')
        print(ind)
        print('iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii')
        cmat=bestlimite_LogReg1(test,train,var,delta=delta,begin=begin,end=end)[0][0] #vamos, pq sim, escolher a primeira melhor confmatrix
        lastz[0][1]+=cmat[0][1]
        lastz[0][0]+=cmat[0][0]
        lastz[1][1]+=cmat[1][1]
        lastz[1][0]+=cmat[1][0]
    return [lastz,pred_percent(lastz,var),sensitivity(lastz,var),specificity(lastz,var)]

In [7]:
#oi4=oi.drop(oi.columns[:-2],axis=1)
#train1=oi4[10:]
#teste1=oi4[0:10]

In [211]:
pre_cross_validation(oi4,0,delta=0.02,begin=0.5,end=0.7)

119
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 110, 111, 112, 113, 114, 115, 116, 117, 118]
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
0.75 max_percent
_______________________________________________________
[array([[9, 3],
       [3, 4]]), 0.64] best pred_percents
-------------------------------------------------------
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 110, 111, 112, 113, 114, 115, 116, 117, 118]
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
0.6666666666666666 max_percent
_______________________________________________________
[array([[12,  6],
       [ 0,  1]]), 0.54] best pred_percents
-------------------------------------------------------
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 110, 111, 112, 113, 114, 115, 116, 117, 118]
iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
0.8888888888888888 max_percent
_______________________________________________________
[array([[8, 1],
       [7, 3]

[array([[80, 31],
        [44, 35]]), 0.7207207207207207, 0.6451612903225806, 0.5303030303030303]

In [151]:
#devolve os coefs da regressão linear com o modelo do Python

def LinReg_check(X,y):
    print('X pode ser uma panda DataFrame, com as variáveis indep')
    from sklearn.linear_model import LinearRegression
    reg=LinearRegression().fit(X,y)
    return reg.intercept,'intercept',reg.coef_,'coefs'

#Em princípio não vamos usar esta fórmula
#Só servirá para comparar os coeficientes com o modelo a formular

In [196]:
#logit False, adapta as variáveis à função grad. descend. 
#logit True, adapta as variáveis à grad. descen. logit

def adapt_X(df,logit=False):
    #print('adapt_X só funciona se a var. indep. estiver na última coluna da DataFrame')
    limit=len(df.columns)-1
    data=df[df.columns[0:limit]]
    depend=np.array([df[df.columns[limit]]],dtype=int)
    first=np.ones((len(data.index),1))
    end=[]
    for i in range(0,len(data.columns)):
        q=[]
        count=np.array([[data.iloc[0][i]]])
        for n in range(1,len(data.index)):
            a=np.array([[data.iloc[n][i]]])
            count=np.vstack([count,a])
        end+=[count]
    bolas=np.array(end[0])
    for aq in range(1, len(end)):
        bolas=np.c_[bolas,end[aq]]
    final=np.c_[first,bolas]
    #return (final,depend)
    if logit:
        return (final,depend)
    else:
        deplin=np.array([depend[0][0]])
        for it in range(1,len(depend[0])):
            #print(deplin)
            deplin=np.vstack([deplin,np.array([[depend[0][it]]])])
        return (final,deplin)

In [153]:
#m é a confusion matrix, ind é o indíce da variável que queremos testar
#ind começa em 0
def sensitivity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    x=m[ind][ind]
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[n][ind]]
            n+=1
    y=sum(q)
    return (x/(x+y))

def specificity(m,ind):
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    print('O índice começa em 0')
    n=0
    q=[]
    while n<len(m):
        if n==ind:
            n+=1
        else:
            q+=[m[ind][n]]
            n+=1
    x=sum(q)
    f=[]
    i=0
    while i<len(m):
        c=0
        if i==ind:
            i+=1
        else:
            while c<len(m):
                if c==ind:
                    c+=1
                else:
                    f+=[m[i][c]]
                    c+=1
            i+=1
    y=sum(f)
    #print(q,f)  #verifica se os valores q estamos a escolher são os certos
    return (y/(x+y))

#A percentagem de previsões corretas feitas pelo modelo
#i é a linha da confusion matrix (o outcome que queremos ver a percentagem de previsões corretas)
def pred_percent(m,i):
    #print(m)
    if not(len(m)==len(m[0])):
        return 'Erro! A matriz tem ser ser quadrada'
    #print('O índice começa em 0')
    x=m[i][i]
    soma=sum(m[i])
    if soma==0:
        return 'Não previu nada para esta variável'
    else:
        return (x/soma)
    

# Resto
# ------------------------------------------------------------------------

In [105]:
oi=pd.read_excel('19_20.xlsx',1)

In [170]:
len(oi.index)//10

11

In [267]:
oi2=pd.concat([oi[0:3],oi[5:8]],sort=False)
ind=list(oi2.index.values)

In [281]:
oi3=oi.drop(oi.columns[:-2],axis=1)
#oi3

In [119]:
def cross_validation(input_,panda_df,var,folds=10,delta=0.01,begin=0.05,end=0.95):
    g=len(panda_df.index)
    print(g)
    assert g>=folds
    extra=g-(g//folds)
    lastz=np.array([[0,0],[0,0]])
    for i in range(folds):
        test=pd.concat([input_[(0+i*folds):(folds+i*folds)],input_[(-extra if extra!=0 else 0):]],sort=false)
        ind=list(test.index.values)
        train=input_.drop(ind)
        cmat=bestlimite_LogReg1(input_,panda_df,var,delta=delta,begin=begin,end=end)[0][0] #vamos, pq sim, escolher a primeira melhor confmatrix
        lastz[0][1]+=cmat[0][1]
        latsz[0][0]+=cmat[0][0]
        lastz[1][1]+=cmat[1][1]
        lastz[1][0]+=cmat[1][0]
    return [lastz,pred_percent(lastz,var),sensitivity(lastz,var),specificity(lastz,var)]